In [1]:
# NEEDED

import re
from functools import lru_cache
import pandas as pd
import numpy as np

def all_algo(pers, basket):
    res = find_recipe(pers, basket) + "\n\n" + find_missing_recipes(pers, basket)
    return res

#GOOD et OPTI
def replace_zeros(dataframe):
    # Convertir toutes les cellules du DataFrame en chaînes de caractères
    dataframe = dataframe.astype(str)

    # Enlever les espaces inutiles de chaque cellule
    dataframe = dataframe.apply(
        lambda col: col.str.strip() if col.dtype == "object" else col
    )

    # Remplacer 'nan' (NaN en tant que chaîne de caractères) par '0'
    dataframe.replace('nan', '0', inplace=True)

    # Remplacer 0 et 0.0 par '0' (comme chaîne de caractères)
    dataframe.replace(['0', '0.0'], '0', inplace=True)
    
    return dataframe


#GOOD et OPTI
@lru_cache(maxsize=None)
def data_pers(pers):
    # Construire le nom du fichier en fonction du nombre de personnes
    nom_fichier = f'Data/test{pers}_eng.csv'
    # Charger le DataFrame et appliquer remplacer_zeros
    df = replace_zeros(pd.read_csv(nom_fichier, sep=';'))
    
    return df

dish_data = pd.read_csv('Data/Dish_eng.csv', sep=';')

#GOOD et OPTI
def find_recipe(pers, basket):
    df = data_pers(pers)
    d_recipes = {row: row for row in df['Dish'] if verify_items(df, basket, row)}

    if d_recipes:
        resultat = "Suggestion: With what you have in your basket you can make :\n"
        resultat += "\n".join(f'- {dish_data.loc[dish_data["Dish"] == recipe, "Sing"].iloc[0]} {recipe}'
                              for recipe in d_recipes)
        return resultat
    else:
        return "With what you have in your basket, you're not in a position to make specific dishes or desserts."

            
#GOOD et OPTI
def extract_items(dish, data):
    # Trouver la ligne correspondant au dish spécifié
    line_dish = data.loc[data['Dish'] == dish]

    # Gérer le cas où aucune ligne correspondante n'est trouvée
    if line_dish.empty:
        return {}

    # S'assurer que line_dish est une Série
    if isinstance(line_dish, pd.DataFrame):
        line_dish = line_dish.iloc[0]

    # Utiliser une compréhension de dictionnaire pour extraire les ingrédients et leurs quantités
    ingredients = {ingredient: quantity 
                   for ingredient, quantity in line_dish.items()
                   if ingredient not in ['Dish', 'No. people'] and quantity != '0'}

    return ingredients

# basket = {'alim1': 'quantite et unity', 'alim2': 'quantunity', etc}

#GOOD et OPTI
def is_subset(dic, basket):
    return set(dic.keys()).issubset(set(basket.keys()))

def clasify(dic):
    for cle, value in dic.items():
        parts = str(value).split()
        quantity = float(parts[0]) if parts[0] else 0
        unit = parts[1] if len(parts) > 1 else ''

        if unit == 'g':
            quantity /= 1000  # Convertir en kg
            unit = 'kg'
        elif unit == 'cl':
            quantity /= 100  # Convertir en litres
            unit = 'l'
        elif unit == 'ml':
            quantity /= 1000  # Convertir en litres
            unit = 'l'
        
        # Mettre à jour le dictionnaire avec la nouvelle valeur
        dic[cle] = f'{quantity} {unit}'

    return dic
#OPTI
def verify_items(data, basket, dish):
    dic = extract_items(dish, data)
    basket = clasify(basket)

    if not is_subset(dic, basket):
        return False

    n = 0
    for cle, value in dic.items():
        if cle in basket:
            basket_value = str(basket[cle]).split()
            dic_value = str(value).split()

            # Gérer les cas où l'unité de mesure n'est pas spécifiée
            if len(basket_value) == 1:
                basket_quantity, basket_unit = float(basket_value[0]), ''
            else:
                basket_quantity, basket_unit = float(basket_value[0]), basket_value[1]

            if len(dic_value) == 1:
                dic_quantity, dic_unit = float(dic_value[0]), ''
            else:
                dic_quantity, dic_unit = float(dic_value[0]), dic_value[1]

            # Vérifier si la quantité et l'unité correspondent
            if (basket_unit == dic_unit and basket_quantity >= dic_quantity) or (dic_unit == 'u' and basket_quantity > 0):
                n += 1

    return n == len(dic)


def duplicate_item(d_recipe, keys, values, item_data, dish_data, i, j):
    recipes = keys[i:i+j]
    item = d_recipe[recipes[0]]
    #print(item)
    
    sin_aliment = item_data.loc[item_data['Aliment'] == item, 'Sing'].iloc[0]
    sins = [dish_data.loc[dish_data['Dish'] == recipe, 'Sing'].iloc[0] for recipe in recipes]

    if j == 1:
        result = f"- You need {sin_aliment} '{item}' to cook {sins[0]} {recipes[0]} \n"
    else:
        dish_sins = ", ".join(f"{sin} {recipe}" for sin, recipe in zip(sins, recipes))
        result = f"- You need {sin_aliment} '{item}' to cook {dish_sins} \n"

    return result


def missing_item(data, basket, dish):
    basket = clasify(basket)
    dic = extract_items(dish, data)
    if not dic:
        #print('A')
        return None

    dico = {}
    nb_aliments_manquants = 0
    total_aliments = len(dic)

    for cle, value in dic.items():
        basket_value = str(basket.get(cle, "0")).split()
        basket_quantity = float(basket_value[0]) if basket_value[0] else 0
        basket_unit = basket_value[1] if len(basket_value) > 1 else ''

        dic_value = str(value).split()
        dic_quantity = float(dic_value[0]) if dic_value[0] else 0
        dic_unit = dic_value[1] if len(dic_value) > 1 else ''

        # Gestion des cas spéciaux '1 u'
        if basket_unit == 'u':
            if dic_unit != 'u' and dic_quantity > 0:
                dico[cle] = f"{dic_quantity} {dic_unit}"
                #print('1')
                nb_aliments_manquants += 1
            continue

        if dic_unit == 'u':
            if basket_quantity > 0:
                continue  # L'ingrédient est présent dans le panier en quantité suffisante
            else:
                dico[cle] = "1 u"
                #print('2')
                nb_aliments_manquants += 1
                continue
        else:
            if basket_quantity == 0:
                dico[cle] = str(dic_quantity) + ' ' + dic_unit
                nb_aliments_manquants += 1
                continue
        
                


        if basket_quantity < dic_quantity:
            if basket_unit == dic_unit:
                qte_manquante = round(dic_quantity - basket_quantity, 3)

                if dic_unit == 'kg' and qte_manquante < 1:
                    qte_manquante *= 1000
                    dico[cle] = f"{qte_manquante} g"
                elif dic_unit == 'l':
                    if qte_manquante < 1 and qte_manquante > 0.1:
                        qte_manquante *= 100
                        dico[cle] = f"{qte_manquante} cl"
                    elif qte_manquante < 0.1:
                        qte_manquante *= 1000
                        dico[cle] = f"{qte_manquante} ml"
                    else:
                        dico[cle] = f"{qte_manquante} l"
                else:
                    dico[cle] = f"{qte_manquante} {dic_unit}"
                #print('3')
                nb_aliments_manquants += 1
            else:
                #print('4')
                nb_aliments_manquants += 1
                #dico[cle] = f"{basket_unit} vs {dic_unit} / {basket_value} vs {dic_value} from {dish} et aliment : {cle}"

    if nb_aliments_manquants == 1 or (nb_aliments_manquants != 0 and nb_aliments_manquants <= total_aliments * 0.2):
        return dico, dish
    else:
        #print('B')
        return None




item_data = pd.read_csv('Data/Item_eng.csv', sep=',')

def find_missing_recipes(pers, basket):
    df = data_pers(pers)
    suggestions = []

    for dish in df['Dish']:
        resultat = missing_item(df, basket, dish)
        if resultat:
            manquants, name_dish = resultat
            #print(resultat)
            if len(manquants) == 1:  # Si un seul ingrédient manque
                item, quantite = list(manquants.items())[0]
                quantite_parts = quantite.split()

                if len(quantite_parts) > 1:  # Si la quantité a une unité de mesure
                    #print(item, '\n\n')
                    #print(quantite_parts)
                    quantite_val, unit = quantite_parts
                    if unit == 'u':
                        d1 = dish_data.loc[dish_data["Dish"] == name_dish, "Sing"].iloc[0]
                        determinant = item_data.loc[item_data['Item'] == item, 'Sing2'].iloc[0]
                        suggestion = f"- You need '{item}' to cook {d1} '{name_dish}'\n"
                    else:
                        #print(item)
                        #print(name_dish)
                        d1 = dish_data.loc[dish_data["Dish"] == name_dish, "Sing"].iloc[0]
                        determinant = item_data.loc[item_data['Item'] == item, 'Sing1'].iloc[0]
                        suggestion = f"- You need {quantite} {determinant} '{item}' to cook {d1} '{name_dish}'\n"
                else:  # Si la quantité est juste un nombre
                    #print(item)
                    d1 = dish_data.loc[dish_data["Dish"] == name_dish, "Sing"].iloc[0]
                    suggestion = f"- You need {quantite}'{item}' to cook {d1} '{name_dish}'\n"
                suggestions.append(suggestion)
            elif len(manquants) == 2:
                alim1, quantite1 = list(manquants.items())[0]
                quantite_parts1 = quantite1.split()
                alim2, quantite2 = list(manquants.items())[1]
                quantite_parts2 = quantite2.split()

                if len(quantite_parts1) > 1:
                    quantite_val1, unit1 = quantite_parts1
                    if len(quantite_parts2) > 1:
                        quantite_val2, unit2 = quantite_parts2
                        if unit1 == 'u' and unit2 == 'u':
                            #print('1')
                            d1 = dish_data.loc[dish_data["Dish"] == name_dish, "Sing"].iloc[0]
                            determinant1 = item_data.loc[item_data['Item'] == alim1, 'Sing2'].iloc[0]
                            determinant2 = item_data.loc[item_data['Item'] == alim2, 'Sing2'].iloc[0]
                            suggestion = f"- You need '{alim1}' and '{alim2}' to cook {d1} '{name_dish}'\n"
                        elif unit1 == 'u':
                            #print('2')
                            d1 = dish_data.loc[dish_data["Dish"] == name_dish, "Sing"].iloc[0]
                            determinant1 = item_data.loc[item_data['Item'] == alim1, 'Sing2'].iloc[0]
                            determinant2 = item_data.loc[item_data['Item'] == alim2, 'Sing1'].iloc[0]
                            suggestion = f"- You need {quantite2} {determinant2} '{alim2}' and '{alim1}' to cook {d1} '{name_dish}'\n"
                        elif unit2 == 'u':
                            #print('3')
                            d1 = dish_data.loc[dish_data["Dish"] == name_dish, "Sing"].iloc[0]
                            determinant1 = item_data.loc[item_data['Item'] == alim1, 'Sing1'].iloc[0]
                            determinant2 = item_data.loc[item_data['Item'] == alim2, 'Sing2'].iloc[0]
                            suggestion = f"- You need {quantite1} {determinant1} '{alim1}' and '{alim2}' to cook {d1} '{name_dish}'\n"
                        else:
                            #print('4')
                            #print(name_dish)
                            d1 = dish_data.loc[dish_data["Dish"] == name_dish, "Sing"].iloc[0]
                            determinant1 = item_data.loc[item_data['Item'] == alim1, 'Sing1'].iloc[0]
                            determinant2 = item_data.loc[item_data['Item'] == alim2, 'Sing1'].iloc[0]
                            suggestion = f"- You need {quantite1} {determinant1} '{alim1}' and {quantite2} {determinant2} '{alim2}' to cook {d1} '{name_dish}'\n"
                    else:
                        if unit1 == 'u':
                            #print('5')
                            d1 = dish_data.loc[dish_data["Dish"] == name_dish, "Sing"].iloc[0]
                            determinant1 = item_data.loc[item_data['Item'] == alim1, 'Sing2'].iloc[0]
                            suggestion = f"- You need {quantite2}'{alim2}' and '{alim1}' to cook {d1} '{name_dish}'\n"
                        else:
                            #print('6')
                            d1 = dish_data.loc[dish_data["Dish"] == name_dish, "Sing"].iloc[0]
                            determinant1 = item_data.loc[item_data['Item'] == alim1, 'Sing1'].iloc[0]
                            suggestion = f"- You need {quantite2}'{alim2}' and {quantite1} {determinant1} '{alim1} to cook {d1} '{name_dish}'\n"
                elif len(quantite_parts2) > 1:
                    quantite_val2, unit2 = quantite_parts2
                    if unit2 == 'u':
                        #print('7')
                        d1 = dish_data.loc[dish_data["Dish"] == name_dish, "Sing"].iloc[0]
                        determinant2 = item_data.loc[item_data['Item'] == alim2, 'Sing2'].iloc[0]
                        suggestion = f"- You need {quantite1}'{alim1}' and '{alim2}' to cook {d1} '{name_dish}'\n"
                    else:
                        #print('8')
                        d1 = dish_data.loc[dish_data["Dish"] == name_dish, "Sing"].iloc[0]
                        determinant2 = item_data.loc[item_data['Item'] == alim2, 'Sing1'].iloc[0]
                        suggestion = f"- You need {quantite1} of '{alim1}' and {quantite2} {determinant2} '{alim2} to cook {d1} '{name_dish}'\n"
                else:
                    #print('9')
                    d1 = dish_data.loc[dish_data["Dish"] == name_dish, "Sing"].iloc[0]
                    suggestion = f"- You need {quantite1}'{alim1}' and {quantite2} '{alim2}' to cook {d1} '{name_dish}'\n"
                suggestions.append(suggestion)
    if suggestions:
        return f"Suggestions of recipes for {pers} person(s):\n" + "".join(suggestions)
    else:
        return "None suggestion is available."

In [2]:
# NEEDED

import pandas as pd
import re
from collections import Counter

def extraire_unites(cellule):
    pattern = r"\b(kg|l|u|slices?|bunches?|leafs?|sprigs?|boxs?|sprigs?|bags?|pods?)\b"
    match = re.search(pattern, str(cellule), re.IGNORECASE)
    return match.group() if match else None

def classer_colonnes_par_unite(df):
    colonnes_par_unite = {"unknow": [], "value": [], "kg": [], "l":[], "slice":[], "bunch":[], "bunch1":[], "leaf":[], "sprig":[], "box":[], "sprig1":[], "bag":[], "pod":[]}

    for col in df.columns[2:]:
        unites = df[col].dropna().apply(extraire_unites)
        
        # Filtrer 'None' et 'u', compter les occurrences des autres unités
        unites_filtrees = [u for u in unites if u and u.lower() != 'u']
        comptage_unites = Counter(unites_filtrees)

        # Compter les cellules contenant uniquement des valeurs numériques non nulles sans unité de mesure
        valeurs_chiffres = df[col].apply(lambda x: (isinstance(x, (int, float)) and x != 0) and extraire_unites(x) is None).sum()

        # Déterminer la catégorie de la colonne
        if valeurs_chiffres > 0 and len(comptage_unites) == 0:
            colonnes_par_unite["value"].append(col)
        elif len(comptage_unites) == 1:
            unite = list(comptage_unites)[0].lower()
            colonnes_par_unite[unite].append(col)
        else:
            colonnes_par_unite["unknow"].append(col)

    return colonnes_par_unite

# Lire les données depuis un fichier CSV
df = pd.read_csv('Data/test3_eng.csv', sep=';')


# Classer les colonnes
colonnes_classees = classer_colonnes_par_unite(df)

#for unite, cols in colonnes_classees.items():
    #print(f"Unité: {unite}, Colonnes: {cols}")


In [3]:
# NEEDED

def set_all_checkboxes(state: bool):
    for checkbox, combobox, item in all_checkboxes:  # Notez que nous utilisons la décomposition du tuple ici
        if state:
            checkbox.select()
        else:
            checkbox.deselect()

def check_cart(cart, qties):
    for item, qty in zip(cart, qties):
        if item in widgets_mapping:
            checkbox, combobox = widgets_mapping[item]
            checkbox.select()
            combobox.set(qty)

def check_cart_1():
    cart_1  = ["Garlic", "Parsley bouquet", "Fresh cream", "Pork tenderloin", "Olive oil", "Egg yolk(s)", "Old-fashioned mustard", "Pepper", "Pâte feuilletée", "Salt"]
    qties_1 = ["3 pod(s)", "4", "500 g", "1 kg", "1 l", "12", "300 g", "150 g", "4", "150 g"]
    check_cart(cart_1, qties_1)
    #for i in range(len(panier_1)):
    #    for checkbox, combobox, item in all_checkboxes:
    #        if panier_1[i] == item:
    #            checkbox.select()
    #            combobox.set(qties[i])

def check_cart_2():
    cart_2  = ['Bay leaf(s)', 'Dry white wine', 'Garlic', 'Olive oil', 'Onion(s)', 'Pepper', 'Pitted prune(s)', 'Potato(s)', 'Rabbit', 'Salt', 'Thyme', 'Chicken bouillon cube(s)', 'Garlic', 'Grated Parmesan', 'Olive oil', 'Paprika', 'Pepper', 'Pumpkin', 'Salt', 'Strong chorizo', 'Carotte(s)', 'Egg(s)', 'Garlic', 'Ginger powder', 'Olive oil', 'Onion(s)', 'Red onion(s)', 'Rice leaves', 'Rice vermicelli', 'Soy sauce', 'Soybeans', 'Bell pepper(s)', 'Carotte(s)', 'Coconut milk', 'Garlic', 'Ginger', 'Green curry paste', 'Oil', 'Olive oil', 'Onion(s)', 'Quinoa', 'Salt', 'Zucchini(s)', 'Almond powder', 'Mash mango', 'Mash passion', 'Powdered sugar', 'Raspberry puree', 'Sugar', 'White egg(s)', 'Black pepper', 'Lenses', 'Olive oil', 'Salt', 'Tomato(s)', 'Apple(s)', 'Egg(s)', 'Flour', 'Liquid vanilla', 'Milk', 'Prune(s)', 'Salt', 'Sugar', 'Yeast', 'Banana(s)', 'Honey', 'Oat flakes', 'Agave syrup', 'Pistachios', 'Tahini']
    qties_2 = ['6.0', '0.4 l', '0.1 kg', '0.1 l', '6', '1 u', '30.0', '12.0', '2.25 kg', '1 u', '1 string(s)', '6.0', '0.1 kg', '0.2 kg', '0.1 l', '1 u', '1 u', '6.0', '1 u', '0.2 kg', '6.0', '6.0', '0.1 kg', '1 u', '0.09 l', '6', '6.0', '18.0', '0.1 kg', '0.1 kg', '0.1 kg', '6', '6.0', '0.6 l', '0.1 kg', '1 u', '1 u', '1 u', '1 u', '6', '0.3 kg', '1 u', '6.0', '0.1 kg', '0.2 kg', '0.2 kg', '0.03 kg', '0.16 kg', '0.1 kg', '6.0', '1 u', '0.5 kg', '0.02 l', '1 u', '6.0', '6.0', '6.0', '0.2 kg', '0.06 l', '0.5 l', '24', '1 u', '0.2 kg', '1 bag(s)', '6.0', '1 u', '0.18 kg', '0.126 l', '0.09 kg', '0.27 kg']
    check_cart(cart_2, qties_2)
    #for i in range(len(panier_2)):
    #    for checkbox, combobox, item in all_checkboxes:
    #        if panier_2[i] == item:
    #            checkbox.select()
    #            combobox.set(qties[i])

def check_cart_3():
    cart_3 = ['Pork tenderloin', 'Poultry', 'Lemon juice', 'Garlic', 'Red beet(s)', 'Carottes', 'Paris mushrooms', 'Zucchini(s)', 'Green bean(s)', 'Coral lentils', 'Green lentils', 'Turnip(s)', 'Onion(s)', 'Red bell pepper(s)', 'Ras-el-hanout', 'Shallot(s)', 'Parsley bouquet', 'Cinnamon', 'Coriander', 'Cumin', 'Turmeric', 'Ginger powder', 'Paprika', 'Pepper', 'Salt', 'Fish', 'Pâte feuilletée', 'Pâte(s) brisée(s)', 'Whipping cream', 'Fresh cream', "Goat's cheese", 'Milk', 'Butter', 'Bitter cocoa powder', 'Cocoa powder', 'White chocolate', 'Flour', 'Gelatin', 'Yeast', 'Baking powder', 'Honey', 'Sugar', 'Crystal sugar', 'Powdered sugar', 'Semolina sugar', 'Vanilla sugar', 'Liquid vanilla', 'Soluble coffee', 'Breadcrumbs', 'Corn starch', 'Oil', 'Olive oil', 'Frying oil', 'Egg yolk(s)', 'Maïzena', 'Egg(s)', 'Old-fashioned mustard', 'Soy sauce', 'Cherries', 'Prune(s)', 'Walnut kernels', 'Sesame seeds', 'Walnuts', 'Nutmeg']
    qties_3 = ['1 kg', '800 g', '1 l', '6 pod(s)', '8', '0.75 kg', '50 g', '6', '0.75 kg', '150 g', '150 g', '6', '6', '3.0', '50 g', '3', '1', '100 g', '3 bunch(es)', '100 g', '50 g', '50 g', '50 g', '150 g', '150 g', '1.2 kg', '2.0', '2.0', '800 g', '1.2 kg', '250 g', '1.5 l', '200 g', '180 g', '250 g', '450 g', '800 g', '6 leaf(s)', '2 bag(s)', '6 bag(s)', '150 g', '500 g', '400 g', '1.35 kg', '300 g', '4 bag(s)', '6', '100 g', '100 g', '60 g', '1 l', '1 l', '1 l', '12', '100 g', '24', '300 g', '50 cl', '1 kg', '30', '50 g', '100 g', '100 g', '130 g']
    #cart_3  = ["Bitter cocoa powder", "Soluble coffee", "White chocolate", "Whipping cream", "Fresh cream", "Flour", "Gelatin", "Egg yolk(s)", "Milk", "Baking powder", "Powdered sugar", "Semolina sugar", "Liquid vanilla", "Egg(s)", "Garlic", "Cinnamon", "Walnut kernels", "Paris mushrooms", "Coriander", "Cumin", "Corn starch", "Olive oil", "Coral lentils", "Green lentils", "Noix de cajou", "Shallot(s)", "Parsley bouquet", "Carottes", "Zucchini(s)", "Turmeric", "Ginger powder", "Green bean(s)", "Turnip(s)", "Onion(s)", "Paprika", "Prune(s)", "Ras-el-hanout", "Salt"]
    #qties_3 = ["180 g", "100 g", "450 g", "800 g", "1.2 kg", "800 g", "6 leaf(s)", "12", "1.5 l", "6 bag(s)", "1.35 kg", "300 g", "6", "24", "6 pod(s)", "100 g", "50 g", "50 g", "3 bouquet(s)", "100 g", "60 g", "1 l", "150 g", "150 g", "132", "3", "1", "0.75 kg", "6", "50 g", "50 g", "0.75 kg", "6", "6", "50 g", "30", "50 g", "150 g"]
    check_cart(cart_3, qties_3)
    #for i in range(len(panier_3)):
    #    for checkbox, combobox, item in all_checkboxes:
    #        if panier_3[i] == item:
    #            checkbox.select()
    #            combobox.set(qties[i])

def selection_changed():
    selected_item = comboboxe.get()
    if selected_item == "No. people.":
        return 1
    else:
        selected_item = int(selected_item)
        return selected_item

def get_checked_items():
    checked_items = {}  # Liste pour stocker les éléments cochés
    for checkbox, combobox, item in all_checkboxes:
        if checkbox.get() == 1:  # Si la case à cocher est cochée
            res = combobox.get()
            checked_items[item] = res  # Ajouter le texte de la case à la liste
    return checked_items

def ouvrir_nouvelle_fenetre():
    basket = get_checked_items()
    print(basket)
    alims = list(basket.keys())
    qties = list(basket.values())
    pers = selection_changed()
    recap = f" - {qties[0]} {alims[0]}\n"
    for i in range(1, len(alims)):
        recap += f" - {qties[i]} {alims[i]}\n"

    nouvelle_fenetre = ctk.CTkToplevel(app)  # Assurez-vous que "app" est la fenêtre principale de votre application
    nouvelle_fenetre.geometry("900x600")
    nouvelle_fenetre.title("Summary of your shopping cart")

    # Créer un CTkScrollableFrame
    scrollable_frame = ctk.CTkScrollableFrame(nouvelle_fenetre)
    scrollable_frame.pack(fill="both", expand=True, padx=20, pady=20)

    # Ajouter le titre du panier
    label_titre = ctk.CTkLabel(scrollable_frame, text="Here's your shopping cart:", anchor="w", justify=ctk.LEFT)
    label_titre.pack(pady=10)

    # Ajouter le récapitulatif des items cochés
    label_recap = ctk.CTkLabel(scrollable_frame, text=recap, anchor="w", justify=ctk.LEFT)
    label_recap.pack(side="top", fill="both", expand=True)

    # Ajouter ce qu"affiche la fonction all() des items cochés
    texte = all_algo(pers, basket)
    label_all = ctk.CTkLabel(scrollable_frame, text=texte, anchor="w", justify=ctk.LEFT)
    label_all.pack(side="top", fill="both", expand=True)

    # Afficher la nouvelle fenêtre
    nouvelle_fenetre.mainloop()

def on_combobox_click(event):
    # Cette fonction sera appelée quand le CTkComboBox est cliqué (focus)
    if combobox.get() == "Select an item":
        combobox.set("")  # Effacez le texte de l"espace réservé

def on_combobox_focusout(event):
    # Cette fonction sera appelée quand le focus est perdu
    if combobox.get() == "":
        combobox.set("Select an item")  # Remettez le texte de l"espace réservé

"""def create_combobox_callback(checkbox):
    # Coche la checkbox lorsque quelque chose est sélectionné dans le combobox
    checkbox.select()"""

def create_combobox_callback(checkbox):
    def combobox_selected(event):
        # Obtenir la valeur actuelle du combobox
        selected_value = event.widget.get()
        
        # Cocher la checkbox si la valeur sélectionnée est différente de "\"
        if selected_value != "//":
            checkbox.select()
        else:
            checkbox.deselect()
    return combobox_selected




In [5]:
# NEEDED

import customtkinter as ctk

# Initialize the main application window
app = ctk.CTk()
app.title("Shopping List")

def find_table_containing_string(dictionnaire_tableaux, string_a_chercher):
    for nom_tableau, tableau in dictionnaire_tableaux.items():
        if string_a_chercher in tableau:
            return nom_tableau
    return "Not find"


item_units = {
    "u_kg": ["Dried apricots", "Agar-agar", "Amandes concassées", "Amandes en poudre vahiné", "Amandes", "Slivered almonds", "Vanilla slivered almonds", "Colliers d'agneaux", "Powdered almonds", "Pineapple", "Chopped dill", "Lemon vanilla flavor", "White asparagus", "Beet(s)", "Red beet(s)", "Butter", "Peanut butter", "Semi-salted butter", "Soft butter", "Salted butter", "Soft butter", "Biscuits", "Spoon cookies", "Bisques de homard", "Chicken breast(s)", "Chard(s)", "Boursin(s)", "Blue Bresse", "Brioche", "Burrata", "Beef", "Ground beef", "Cacahuètes", "Bitter cocoa", "Cocoa powder", "Unsweetened cocoa powder", "Soluble coffee", "Duck(s)", "Cancoillotte", "Cinnamon", "Brown sugar", "Cherries", "Walnut kernels", "Sausage meat", "Mushrooms", "Paris mushrooms", "Chanterelle(s)", "Chantilly", "Breadcrumbs", "Chocolate", "Bitter chocolate powder", "Milk chocolate", "White chocolate", "Dark chocolate", "50% dark chocolate", "Dark pastry chocolate", "70% cocoa dark chocolate", "Dark baking chocolate", "Pastry chocolate", "Chorizo", "Strong chorizo", "Brussels sprouts", "Green cabbage", "Cabbage(s)", "Chou(x) de bruxelle", "Cauliflower", "Chestnut(s)", "Goat cheese", "Heart of palm", "Quince", "Lamb collar", "Apple compote", "Comté cheese", "Shredded Comté cheese", "Tomato concentrate", "Conchiglionis", "Raspberry jam", "Chocolate chips", "Parmesan shavings", "Hulls", "Corn flakes", "Raspberry sauce", "Red fruit coulis", "Tomato coulis", "Butternut squash", "Couscous", "Crab(s)", "Crackers salés", "Dried cranberries", "Cranberrys", "Shrimp(s)", "Shrimp(s)", "Grey shrimps", "Pink shrimps", "Crème", "Light cream", "Custard cream", "Chestnut cream", "Whole cream", "30% full cream", "Whipping cream", "Crème fleurette 30%", "Whipped cream", "Heavy cream", "Fresh cream", "30% fresh cream", "30% fresh cream", "3% light cream", "Whole cream", "Mounted cream", "Sour cream", "Crêpe dentelle(s)", "Cumin", "Turmeric", "Curry", "Capers", "Porcini mushrooms", "Celery root", "Medjool date", "Dates", "Medjool date", "Dried dates", "Diced ham", "Emmental", "Grated Emmental cheese", "Squid(s)", "Fiscella", "Flour", "Buckwheat flour", "Wheat flour", "Rice flour", "Sticky rice flour", "Sticky rice flour", "Flour t55", "Sifted flour", "Feta cheese", "Figs", "Dried figs", "Beef tenderloin", "Pork tenderloin", "Pork tenderloin", "Salmon fillet(s)", "Herbs", "Oat flakes", "Foie gras", "Strawberry(s)", "Tagada strawberry(s)", "Raspberry(s)", "Cheese", "Cottage cheese", "0% fromage blanc", "Goat's cheese", "Feta cheese", "Grated cheese", "Starch", "Corn starch", "Gavotte", "Ginger", "Ginger powder", "Giraumon", "Clove(s)", "Chocolate ice cream", "Gnocchis", "Gorgonzola", "Chia seeds", "Pumpkin seeds", "Cumin seeds", "Sesame seeds", "White sesame seeds", "Golden sesame seeds", "Goose fat", "Coarse salt", "Gruyère", "Grated Gruyère", "Guanciale", "Poultry gizzard(s)", "Green bean(s)", "White beans", "Pink beans", "Kidney beans", "Green bean(s)", "Herbs", "Dried herbs", "Ham", "Cooked ham", "Parma ham", "Jeunes pousses d'épinard", "Ketchup", "Lambis", "Rabbit", "Smoked bacon", "Bacon", "Smoked bacon", "Lenses", "Coral lentils", "Green lentils", "Raw linguine", "Litchi(s)", "Macaronis", "Maïzena", "Mango(s)", "Cassava", "Margarine", "Melted margarine", "Vegetable margarine", "Mascarpone", "Mayonnaise", "Corn", "Maïzena", "McVitties", "Breadcrumbs", "Honey", "Truffle honey", "Mini carottes", "Mirabelles", "Mirin", "Dried morel(s)", "Salt cod", "Mussels", "Mousseron(s)", "Mustard", "Strong mustard", "Old-fashioned mustard", "Mozzarella", "Grated mozzarella", "Nutmeg", "Myrtilles", "4-spice blend", "Nesquik", "Hazelnuts", "Shelled hazelnuts", "Chopped hazelnuts", "Hazelnut powder", "Whole hazelnuts", "Walnuts", "Crushed nuts", "Grated coconut", "Grenoble walnut", "Scallops", "Nutmeg", "Noodles", "Nutella", "Cod eggs", "Yellow onion(s)", "Green olives", "Olives", "Black olives", "Green olives", "Oregano", "Stale bread", "Beef chuck", "Papaya", "Parmesan", "Grated Parmesan", "Sweet potato(s)", "Pecorino", "Pennes", "Chopped parsley", "Chopped parsley", "Chopped parsley", "Petits beurres", "Peas", "Philadelphia", "Pine pinions", "Espelette pepper", "Cayenne pepper", "Sweet pepper powder", "Chilli powder", "Guinea fowl", "Pistachios", "Chopped pistachios", "Unsalted pistachios", "Pizzoccheri", "Oyster mushroom(s)", "Pears in syrup", "Split peas", "Chickpeas", "Fish", "Lamb belly", "Bell pepper(s)", "Bintje potato", "Potatoes", "Pork", "Ground pork", "Pumpkin(s)", "Almond powder", "Cocoa powder", "Yellow curry powder", "Hazelnut powder", "Pistachio powder", "Almond powder", "Pousses d'épinard", "Bamboo shoot", "Soybean pods", "Pralinoise", "Prune(s)", "Dried prunes", "Tomato pulp", "Raspberry puree", "Mash mango", "Mash passion", "Mash tomato", "Hazelnut paste", "Pistachio paste", "Rice paste", "Pasta", "Pépites de chocolat", "Pépites de chocolat noir", "Peaches in syrup", "Quinoa", "Dried grapes", "Ras-el-hanout", "Reblochon", "Leftover meat", "Rhubarb", "Ricotta", "Rigatonis", "Rice", "Arborio rice", "Sticky rice", "Long rice", "Round rice", "Plain puffed rice", "Thaï rice", "Risotto rice", "Roquefort", "Rocket salad", "Brittany shortbreads", "Safran", "Philadelphia", "Sainte-maure de touraine", "Buckwheat", "Savory", "Chocolate sauce", "Soy sauce", "Tomato sauce", "Yakitori sauce", "Smoked sausage(s)", "Sausage", "Salmon", "Cuttlefish", "Saddle of lamb", "Semolina", "Fine semolina", "Shiitake(s)", "Soybeans", "Oat bran", "Soup", "Spaghettis", "Spéculoos", "Sugar", "Brown sugar", "Complete sugar", "Crystal sugar", "Cane sugar", "Powdered sugar", "Powdered sugar", "Brown sugar", "Semolina sugar", "Sesame", "Tagliatelles", "Tahini", "Ham heel", "Tapioca", "Tuna", "Tomate(s) pelée(s)", "Chopped tomatoes", "Dried tomatoes", "Jerusalem artichoke", "Black truffle(s)", "Smoked trout", "Vanilla powder", "Vergeoise blonde", "Chinese vermicelli", "Rice vermicelli", "Meat", "Beef", "Ground beef", "Lean beef", "Minced meat", "Poultry", "Yogurt", "Chopped shallot(s)", "Sweetener", "Pork shoulder", "Chopped spinach", "Spinach(s)", "Dehydrated garlic", "Garlic powder", "Roasted almonds", "Baies roses", "Baies", "Bicarbonate of soda", "Bulot(s)", "Bûches chèvre", "Bitter cocoa powder", "Coffee", "Cinnamon powder", "Caramel", "Salted butter caramel", "Caviar", "Chocolate powder", "Strawberry jam", "Chopped coriander", "Coriander powder", "Corned-beef", "Croutons", "Cumin powder", "Capers", "Spices", "Vanilla essence", "Chopped tarragon", "Vanilla extract", "Poultry fillets", "Edible flowers", "Orange blossom", "Salt flowers", "Red fruit", "Red fruit", "Trim", "Vanilla ice cream", "Fennel seeds", "Mustard seeds", "Harissa", "Provence herbs", "Philadelphia", "Merguez", "Acacia honey", "Morel(s)", "Chocolate mousse", "Nutmeg powder", "Grated nutmeg", "5-berry mix", "Butter hazelnut", "Roasted hazelnuts", "Nutmeg", "Nutmeg powder", "Grated nutmeg", "Nuoc mâm", "Clam(s)", "Paprika", "Smoked paprika", "Japanese pearls", "Truffle beads", "Curled parsley", "Sweet peas", "Pork belly", "Pepper", "White pepper", "Lebanese pepper", "Black pepper", "Porto", "Chicken", "Peanut paste", "Green curry paste", "Raviole dough", "Bolognese sauce", "Nuoc mam sauce", "Sausage(s)", "Salt", "Smen", "Blond sugar", "Grainy sugar", "White sesame", "Tabasco", "Cabbage", "Chocolate vermicelli", "Ground meat 5% (fat)", "Wasabi", "Orange zests"],
    "u_l": ["Beef broth", "Vegetable broth", "Poultry broth", "Coconut cream", "Liquid fresh cream", "Whole cream", "Semi-thick fresh cream", "Heavy cream", "15% liquid cream", "30% cream", "Light cream", "Orange blossom water", "Poultry stock", "Veal stock", "Poultry stock", "Fish stock", "Oil", "Olive oil", "Coconut oil", "Rapeseed oil", "Walnut oil", "Sesame oil", "Sunflower oil", "Vegetable oil", "Orange juice", "Lemon juice", "Milk", "Unsweetened condensed milk", "Almond milk", "Goat's milk", "Coconut milk", "Soy milk", "Semi-skimmed milk", "Whole milk", "Plant milk", "Skim milk", "Teriyaki sauce", "Agave syrup", "Maple syrup", "Liquid vanilla", "White wine", "Dry white wine", "Red wine", "Amaretto", "Armagnac", "Bitter almond aroma", "Vanilla flavor", "Broth", "Cognac", "Colombo", "Food coloring red", "Whipped cream", "Balsamic vinegar cream with toscoro berries", "35% liquid cream", "Thick cream", "Cuttlefish ink", "Yellow food coloring drops", "Red food coloring drop(s)", "Grand marnier", "Peanut oil", "Frying oil", "Lime juice", "Lychee juice", "Grapefruit juice", "Veal juice", "Rhum", "Dark rum", "Vermouth", "Madeira wine", "Yellow wine", "Rosé wine", "Vinegar", "Balsamic vinegar", "White balsamic vinegar", "White vinegar", "Spirit vinegar", "Cider vinegar", "Rice vinegar", "Red wine vinegar", "Sherry vinegar", "Whisky"],
    "u_value": ["Escalopes de dinde", "Escalopes de poulet", "Chicken wing(s)", "Victoria pineapple", "Andouillette(s)", "Eggplant(s)", "Avocado(s)", "Banana(s)", "Tray(s) of button mushrooms", "Cherry tomato tray(s)", "Nice apples", "Thick slices of farmhouse bread", "Spoon cookies", "White egg(s)", "White leek(s)", "Jar of pickled bell pepper(s)", "Vegetable broth cube", "Chicken broth", "Celery stalk(s)", "Brick(s) of liquid crème fraîche", "Brocoli(s)", "Fennel bulb", "Bâtons citronnelle", "Bâtons surimis", "Bâtonnets surimis", "Goat's cheese log", "Camembert", "Carottes", "Fresh square", "Carrés chocolat noir", "Chipolatas", "Mild chorizo", "White cabbage", "Kohlrabi", "Lime(s)", "Clove(s)", "Lamb collar(s)", "Combava", "Cucumber(s)", "Pickles", "Zucchini(s)", "Broth cube(s)", "Beef bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Rabbit legs", "Chicken thighs", "Lamb chop", "Artichoke heart", "Back of fresh salmon", "Endives", "Beef rib steak", "Shoulder(s) of lamb", "Turkey cutlet(s)", "Chicken cutlet(s)", "Star anise", "Fennel", "Brick pastry", "Gelatin sheets", "Lasagne sheet(s)", "Bay leaf(s)", "Filo sheets", "Rice leaves", "White figs", "Smoked tenderloin", "Beef tenderloin", "Tournedos filet", "Pike-perch fillets", "Sole fillets", "Pie crust", "Passion fruit(s)", "Patties", "Rice cakes", "Gambas", "Gouttes de tabasco", "Granny", "Grappes de tomates", "Egg yolk(s)", "Khakies", "Kiwi(s)", "Kubor", "Lobsters", "Langoustines", "Manicotti", "Mackerel(s)", "Maroilles", "Mini snickers", "Pieces of sugar", "Turnip(s)", "White onion(s)", "Sweet onion(s)", "Chopped onion(s)", "Oignon(s) moyen(s)", "Red onion(s)", "Green onion(s)", "Chopped onion(s)", "Pitted black olives", "Orange(s)", "Orange(s)", "Oreos", "Hamburger bun(s)", "Grapefruit", "Parsnips", "Boudoir package(s)", "Marshmallow pack(s)", "Wafer pack(s)", "Pack(s) of bacon", "Package(s) smoked bacon", "Powdered hazelnut pack(s)", "Paupiettes", "Rump steak(s)", "Salmon steak(s)", "Small peppers", "Small green peppers", "Small red peppers", "Small onion(s)", "Small apples", "Small tomatoes", "Pigeon", "Dried pepper", "Green pepper", "Red pepper(s)", "Dragon fruit", "Praline plate(s)", "Pear(s)", "Leek(s)", "Yellow bell pepper(s)", "Red bell pepper(s)", "Green bell pepper(s)", "Apple(s)", "Potato(es)", "Pumpkin", "Octopus(s)", "Pitted prune(s)", "Shortbread dough", "Pizza dough", "Pie dough", "Pâte(s) brisée(s)", "Pêches", "Quenelle(s)", "Monkfish tail(s)", "Raifort", "Goat's cheese log", "Rouleau pâte(s) feuilletée(s)", "Rumsteak(s)", "Scallops", "Pike-perch", "Sardine", "Sausage(s)", "Tuna steak(s)", "Chopped steak(s)", "Guinea fowl supreme", "Capon supremes", "Garlic and parsley tablet(s)", "Milk chocolate tablet(s)", "White chocolate tablet(s)", "Black chocolate tablet(s)", "Rhubarb stems", "Tomato(s)", "Tomate(s) cerise(s)", "Tomate(s) confite(s)", "Dried tomato(s)", "Medium tomatoes", "Corn tortilla", "Tortillas", "Slices of ham", "Slice(s) of Bayonne ham", "Parma ham slice(s)", "Slices of smoked salmon", "Trout(s)", "Twix", "Lemon zests", "Lemon zests", "Shallot(s)", "Minced shallot(s)", "Lamb shoulder", "Egg(s)", "Parsley bouquet", "Garnished bouquet(s)", "Bottle(s) brut champagne", "Bottle(s) red wine", "Can(s) tomato paste", "Lemon(s)", "Crêpes", "Toothpicks", "Onion(s)", "Milk bread", "Chili pepper", "Sweet pepper", "Hot pepper(s)", "Pâte feuilletée", "Garlic sausage"],
    "u_bunch": ["Green asparagus(s)", "Rosemary"],
    "u_slice": ["Bacon", "Baguette", "Cheddar", "Swordfish", "Leg of lamb", "Bread", "Brioche bread", "Gingerbread", "Country bread", "Crumb bread"],
    "u_bunch1": ["Chervil", "Menthe", "Parsley", "Flat-leaf parsley"],
    "u_leaf": ["Basil", "Gelatin", "Lettuce", "Lasagnes", "Laurel", "Fresh mint", "Filo dough", "Salad", "Sauge", "Wraps"],
    "u_sprig": ["Dill", "Thyme", "Chives", "Coriander", "Tarragon"],
    "u_box": ["Flageolet(s)", "Garlic and herb cheese", "Condensed milk", "Sweetened condensed milk", "Litchi in syrup", "Mutti garlic polpa"],
    "u_bag": ["Cream pudding", "Crêpes dentelles", "Sachet(s) de crêpes dentelles", "Instant jelly", "Yeast", "Baker's yeast", "Baking powder", "Frozen vegetable medley", "Vanilla sugar", "Tortellinis"],
    "u_pod": ["Garlic", "Cardamom", "Vanilla"]
}

units_value = {
    "u_kg": ["50 g", "100 g", "150 g", "200 g", "300 g", "400 g", "500 g", "750 g", "1 kg", "1.5 kg", "2 kg", "2.5 kg", "3 kg"],
    "u_l": ["10 ml", "25 ml", "50 ml", "75 ml", "10 cl", "30 cl", "50 cl", "75 cl", "1 l", "1.5 l", "2 l"],
    "u_value": ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30"],
    "u_bunch": ["1 bunch(es)", "2 bunch(es)", "3 bunch(es)", "4 bunch(es)", "5 bunch(es)", "6 bunch(es)", "7 bunch(es)", "8 bunch(es)", "9 bunch(es)", "10 bunch(es)"],
    "u_slice": ["1 slice(s)", "2 slice(s)", "3 slice(s)", "4 slice(s)", "5 slice(s)", "6 slice(s)", "7 slice(s)", "8 slice(s)", "9 slice(s)", "10 slice(s)", "15 slice(s)", "20 slice(s)", "25 slice(s)", "30 slice(s)", "35 slice(s)", "40 slice(s)", "45 slice(s)", "50 slice(s)"],
    "u_bunch1": ["1 bunch(es)", "2 bunch(es)", "3 bunch(es)", "4 bunch(es)", "5 bunch(es)", "6 bunch(es)", "7 bunch(es)", "8 bunch(es)", "9 bunch(es)", "10 bunch(es)"],
    "u_leaf": ["1 leaf(s)", "2 leaf(s)", "3 leaf(s)", "4 leaf(s)", "5 leaf(s)", "6 leaf(s)", "7 leaf(s)", "8 leaf(s)", "9 leaf(s)", "10 leaf(s)", "12 leaf(s)", "14 leaf(s)", "16 leaf(s)", "18 leaf(s)", "20 leaf(s)", "25 leaf(s)", "30 leaf(s)"],
    "u_sprig": ["1 sprig(s)", "2 sprig(s)", "3 sprig(s)", "4 sprig(s)", "5 sprig(s)", "6 sprig(s)", "7 sprig(s)", "8 sprig(s)", "9 sprig(s)", "10 sprig(s)"],
    "u_box": ["1 box(s)", "2 box(s)", "3 box(s)", "4 box(s)", "5 box(s)"],
    "u_sprig": ["1 sprig(s)", "2 sprig(s)", "3 sprig(s)", "4 sprig(s)", "5 sprig(s)"],
    "u_bag": ["1 bag(s)", "2 bag(s)", "3 bag(s)", "4 bag(s)", "5 bag(s)"],
    "u_pod": ["1 pod(s)", "2 pod(s)", "3 pod(s)", "4 pod(s)", "5 pod(s)", "6 pod(s)", "7 pod(s)", "8 pod(s)", "9 pod(s)", "10 pod(s)", "12 pod(s)", "14 pod(s)", "16 pod(s)"]

}


item_categories = [
    ("Meats:", ["Chicken wing(s)", "Andouillette(s)", "Bacon", "Chicken breast(s)", "Beef", "Ground beef", "Duck(s)", "Sausage meat", "Chipolatas", "Chorizo", "Mild chorizo", "Strong chorizo", "Colliers d'agneaux", "Corned-beef", "Rabbit legs", "Chicken thighs", "Lamb chop", "Diced ham", "Beef rib steak", "Shoulder(s) of lamb", "Escalopes de dinde", "Escalopes de poulet", "Beef tenderloin", "Pork tenderloin", "Pork tenderloin", "Smoked tenderloin", "Beef tenderloin", "Poultry fillets", "Foie gras", "Poultry stock", "Veal stock", "Poultry stock", "Leg of lamb", "Goose fat", "Guanciale", "Poultry gizzard(s)", "Ham", "Cooked ham", "Parma ham", "Veal juice", "Rabbit", "Smoked bacon", "Bacon", "Smoked bacon", "Merguez", "Beef chuck", "Clam(s)", "Pack(s) of bacon", "Package(s) smoked bacon", "Paupiettes", "Rump steak(s)", "Pigeon", "Guinea fowl", "Lamb belly", "Pork belly", "Pork", "Ground pork", "Chicken", "Leftover meat", "Rumsteak(s)", "Sausage(s)", "Sausage(s)", "Smoked sausage(s)", "Sausage", "Garlic sausage", "Saddle of lamb", "Chopped steak(s)", "Guinea fowl supreme", "Capon supremes", "Ham heel", "Slices of ham", "Slice(s) of Bayonne ham", "Parma ham slice(s)", "Meat", "Beef", "Ground beef", "Lean beef", "Minced meat", "Ground meat 5% (fat)", "Poultry", "Lamb shoulder", "Pork shoulder"]),
    ("Drinks:", ["Amaretto", "Armagnac", "Bottle(s) brut champagne", "Bottle(s) red wine", "Cognac", "Colombo", "Grand marnier", "Orange juice", "Lemon juice", "Lime juice", "Lychee juice", "Grapefruit juice", "Porto", "Rhum", "Dark rum", "Vergeoise blonde", "Vermouth", "White wine", "Dry white wine", "Madeira wine", "Yellow wine", "Rosé wine", "Red wine", "Whisky"]),
    ("Vegetables:", ["Garlic", "Dehydrated garlic", "Garlic powder", "White asparagus", "Green asparagus(s)", "Eggplant(s)", "Avocado(s)", "Tray(s) of button mushrooms", "Cherry tomato tray(s)", "Beet(s)", "Red beet(s)", "White leek(s)", "Chard(s)", "Jar of pickled bell pepper(s)", "Can(s) tomato paste", "Brocoli(s)", "Carottes", "Mushrooms", "Paris mushrooms", "Chanterelle(s)", "White cabbage", "Brussels sprouts", "Kohlrabi", "Green cabbage", "Cabbage(s)", "Chou(x) de bruxelle", "Cauliflower", "Heart of palm", "Combava", "Tomato concentrate", "Cucumber(s)", "Pickles", "Tomato coulis", "Butternut squash", "Zucchini(s)", "Porcini mushrooms", "Artichoke heart", "Endives", "Flageolet(s)", "Giraumon", "Clove(s)", "Grappes de tomates", "White beans", "Pink beans", "Kidney beans", "Green bean(s)", "Jeunes pousses d'épinard", "Lettuce", "Lenses", "Coral lentils", "Green lentils", "Frozen vegetable medley", "Cassava", "Corn", "Mini carottes", "Mousseron(s)", "Turnip(s)", "Onion(s)", "White onion(s)", "Sweet onion(s)", "Chopped onion(s)", "Yellow onion(s)", "Oignon(s) moyen(s)", "Red onion(s)", "Green onion(s)", "Chopped onion(s)", "Green olives", "Olives", "Black olives", "Pitted black olives", "Green olives", "Parsnips", "Sweet potato(s)", "Truffle beads", "Small peppers", "Small green peppers", "Small red peppers", "Small onion(s)", "Small tomatoes", "Peas", "Chili pepper", "Espelette pepper", "Cayenne pepper", "Sweet pepper", "Sweet pepper powder", "Chilli powder", "Hot pepper(s)", "Dried pepper", "Green pepper", "Red pepper(s)", "Oyster mushroom(s)", "Leek(s)", "Pears in syrup", "Split peas", "Chickpeas", "Sweet peas", "Bell pepper(s)", "Yellow bell pepper(s)", "Red bell pepper(s)", "Green bell pepper(s)", "Pumpkin", "Pumpkin(s)", "Pousses d'épinard", "Bamboo shoot", "Soybean pods", "Tomato pulp", "Mash tomato", "Ras-el-hanout", "Rocket salad", "Salad", "Shiitake(s)", "Soybeans", "Tapioca", "Tomato(s)", "Tomate(s) cerise(s)", "Tomate(s) confite(s)", "Tomate(s) pelée(s)", "Dried tomato(s)", "Chopped tomatoes", "Medium tomatoes", "Dried tomatoes", "Jerusalem artichoke", "Black truffle(s)", "Cabbage", "Shallot(s)", "Chopped shallot(s)", "Minced shallot(s)", "Chopped spinach", "Spinach(s)"]),
    ("Herbs and spices:", ["Dill", "Chopped dill", "Basil", "Parsley bouquet", "Garnished bouquet(s)", "Celery stalk(s)", "Fennel bulb", "Bâtons citronnelle", "Cinnamon", "Cinnamon powder", "Chervil", "Chives", "Clove(s)", "Coriander", "Chopped coriander", "Coriander powder", "Cumin", "Cumin powder", "Turmeric", "Curry", "Celery root", "Spices", "Chopped tarragon", "Fennel", "Bay leaf(s)", "Filo sheets", "Rice leaves", "Herbs", "Edible flowers", "Salt flowers", "Trim", "Ginger", "Ginger powder", "Fennel seeds", "Mustard seeds", "Coarse salt", "Herbs", "Provence herbs", "Dried herbs", "Kubor", "Laurel", "Menthe", "Fresh mint", "4-spice blend", "5-berry mix", "Oregano", "Paprika", "Smoked paprika", "Parsley", "Chopped parsley", "Curled parsley", "Chopped parsley", "Chopped parsley", "Flat-leaf parsley", "Pepper", "White pepper", "Lebanese pepper", "Black pepper", "Yellow curry powder", "Rosemary", "Safran", "Buckwheat", "Savory", "Sauge", "Salt", "Garlic and parsley tablet(s)", "Thyme", "Rhubarb stems", "Wasabi"]),
    ("Fishes:", ["Bisques de homard", "Bulot(s)", "Bâtons surimis", "Bâtonnets surimis", "Caviar", "Crab(s)", "Shrimp(s)", "Grey shrimps", "Pink shrimps", "Back of fresh salmon", "Cuttlefish ink", "Swordfish", "Tarragon", "Tournedos filet", "Salmon fillet(s)", "Pike-perch fillets", "Sole fillets", "Fish stock", "Gambas", "Lambis", "Lobsters", "Langoustines", "Mackerel(s)", "Morel(s)", "Dried morel(s)", "Salt cod", "Mussels", "Scallops", "Nuoc mâm", "Cod eggs", "Salmon steak(s)", "Fish", "Mutti garlic polpa", "Octopus(s)", "Quenelle(s)", "Monkfish tail(s)", "Scallops", "Pike-perch", "Sardine", "Salmon", "Cuttlefish", "Soup", "Tuna steak(s)", "Tuna", "Slices of smoked salmon", "Smoked trout", "Trout(s)"]),
    ("Dough:", ["Peanut paste", "Green curry paste", "Hazelnut paste", "Pistachio paste", "Rice paste", "Pâte feuilletée", "Filo dough", "Shortbread dough", "Pizza dough", "Raviole dough", "Pie dough", "Pâte(s) brisée(s)", "Pasta"]),
    ("Milk item:", ["Boursin(s)", "Blue Bresse", "Brick(s) of liquid crème fraîche", "Burrata", "Goat's cheese log", "Bûches chèvre", "Camembert", "Cancoillotte", "Fresh square", "Chantilly", "Cheddar", "Goat cheese", "Comté cheese", "Shredded Comté cheese", "Parmesan shavings", "Crème", "Light cream", "Custard cream", "Whipped cream", "Coconut cream", "Chestnut cream", "Balsamic vinegar cream with toscoro berries", "Whole cream", "30% full cream", "Whipping cream", "Crème fleurette 30%", "Whipped cream", "Heavy cream", "Fresh cream", "30% fresh cream", "30% fresh cream", "3% light cream", "Whole cream", "Liquid fresh cream", "Whole cream", "Semi-thick fresh cream", "Heavy cream", "15% liquid cream", "35% liquid cream", "30% cream", "Light cream", "Mounted cream", "Cream pudding", "Sour cream", "Thick cream", "Emmental", "Grated Emmental cheese", "Fiscella", "Feta cheese", "Cheese", "Garlic and herb cheese", "Cottage cheese", "0% fromage blanc", "Goat's cheese", "Feta cheese", "Grated cheese", "Chocolate ice cream", "Vanilla ice cream", "Gorgonzola", "Gruyère", "Grated Gruyère", "Philadelphia", "Milk", "Condensed milk", "Unsweetened condensed milk", "Sweetened condensed milk", "Almond milk", "Goat's milk", "Coconut milk", "Soy milk", "Semi-skimmed milk", "Whole milk", "Plant milk", "Skim milk", "Manicotti", "Maroilles", "Mascarpone", "Mozzarella", "Grated mozzarella", "Parmesan", "Grated Parmesan", "Pecorino", "Philadelphia", "Pizzoccheri", "Raifort", "Reblochon", "Ricotta", "Goat's cheese log", "Roquefort", "Philadelphia", "Sainte-maure de touraine", "Yogurt"]),
    ("Bakery:", ["Lemon vanilla flavor", "Bitter almond aroma", "Vanilla flavor", "Butter", "Peanut butter", "Semi-salted butter", "Soft butter", "Salted butter", "Soft butter", "Bicarbonate of soda", "Biscuits", "Spoon cookies", "Bitter cocoa", "Bitter cocoa powder", "Cocoa powder", "Unsweetened cocoa powder", "Caramel", "Salted butter caramel", "Carrés chocolat noir", "Brown sugar", "Chocolate", "Bitter chocolate powder", "Milk chocolate", "White chocolate", "Chocolate powder", "Dark chocolate", "50% dark chocolate", "Dark pastry chocolate", "70% cocoa dark chocolate", "Dark baking chocolate", "Pastry chocolate", "Food coloring red", "Chocolate chips", "Orange blossom water", "Vanilla essence", "Vanilla extract", "Flour", "Buckwheat flour", "Wheat flour", "Rice flour", "Sticky rice flour", "Sticky rice flour", "Flour t55", "Sifted flour", "Gelatin sheets", "Orange blossom", "Yellow food coloring drops", "Red food coloring drop(s)", "Gelatin", "Yeast", "Baker's yeast", "Baking powder", "Margarine", "Melted margarine", "Vegetable margarine", "Honey", "Acacia honey", "Truffle honey", "Mini snickers", "Pieces of sugar", "Nesquik", "Butter hazelnut", "Nutella", "Oreos", "Boudoir package(s)", "Wafer pack(s)", "Petits beurres", "Praline plate(s)", "Almond powder", "Cocoa powder", "Pralinoise", "Pépites de chocolat", "Pépites de chocolat noir", "Brittany shortbreads", "Agave syrup", "Maple syrup", "Smen", "Spéculoos", "Sugar", "Blond sugar", "Brown sugar", "Complete sugar", "Crystal sugar", "Cane sugar", "Powdered sugar", "Powdered sugar", "Grainy sugar", "Brown sugar", "Semolina sugar", "Vanilla sugar", "Milk chocolate tablet(s)", "White chocolate tablet(s)", "Black chocolate tablet(s)", "Twix", "Vanilla", "Vanilla powder", "Liquid vanilla", "Chocolate vermicelli", "Sweetener"]),
    ("Others:", ["White egg(s)", "Broth", "Vegetable broth cube", "Beef broth", "Vegetable broth", "Chicken broth", "Poultry broth", "Cacahuètes", "Coffee", "Soluble coffee", "Breadcrumbs", "Hulls", "Corn flakes", "Crackers salés", "Croutons", "Crêpes dentelles", "Crêpes", "Sachet(s) de crêpes dentelles", "Broth cube(s)", "Beef bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Toothpicks", "Squid(s)", "Brick pastry", "Lasagne sheet(s)", "Oat flakes", "Pie crust", "Tagada strawberry(s)", "Starch", "Corn starch", "Patties", "Rice cakes", "Gavotte", "Instant jelly", "Granny", "Oil", "Peanut oil", "Olive oil", "Coconut oil", "Rapeseed oil", "Frying oil", "Walnut oil", "Sesame oil", "Sunflower oil", "Vegetable oil", "Egg yolk(s)", "Macaronis", "Maïzena", "Maïzena", "McVitties", "Breadcrumbs", "Mirin", "Chocolate mousse", "Bread", "Milk bread", "Brioche bread", "Gingerbread", "Country bread", "Crumb bread", "Stale bread", "Hamburger bun(s)", "Marshmallow pack(s)", "Japanese pearls", "Oat bran", "Corn tortilla", "Tortillas", "Vinegar", "Balsamic vinegar", "White balsamic vinegar", "White vinegar", "Spirit vinegar", "Cider vinegar", "Rice vinegar", "Red wine vinegar", "Sherry vinegar", "Wraps", "Egg(s)"]),
    ("Sides:", ["Baguette", "Thick slices of farmhouse bread", "Brioche", "Conchiglionis", "Couscous", "Gnocchis", "Lasagnes", "Raw linguine", "Noodles", "Pennes", "Bintje potato", "Potato(es)", "Potatoes", "Quinoa", "Rigatonis", "Rice", "Arborio rice", "Sticky rice", "Long rice", "Round rice", "Plain puffed rice", "Thaï rice", "Risotto rice", "Semolina", "Fine semolina", "Spaghettis", "Tagliatelles", "Tortellinis", "Chinese vermicelli", "Rice vermicelli"]),
    ("Sauces:", ["Harissa", "Ketchup", "Mayonnaise", "Mustard", "Strong mustard", "Old-fashioned mustard", "Bolognese sauce", "Chocolate sauce", "Nuoc mam sauce", "Soy sauce", "Teriyaki sauce", "Tomato sauce", "Yakitori sauce", "Tabasco", "Tahini", "Gouttes de tabasco"]),
    ("Fruits:", ["Dried apricots", "Agar-agar", "Pineapple", "Victoria pineapple", "Baies roses", "Baies", "Banana(s)", "Nice apples", "Cardamom", "Cherries", "Lemon(s)", "Lime(s)", "Quince", "Apple compote", "Strawberry jam", "Raspberry jam", "Raspberry sauce", "Red fruit coulis", "Dried cranberries", "Cranberrys", "Capers", "Dates", "Medjool date", "Dried dates", "Star anise", "Figs", "White figs", "Dried figs", "Strawberry(s)", "Raspberry(s)", "Passion fruit(s)", "Red fruit", "Red fruit", "Khakies", "Kiwi(s)", "Litchi in syrup", "Litchi(s)", "Mango(s)", "Mirabelles", "Myrtilles", "Grated coconut", "Orange(s)", "Grapefruit", "Papaya", "Small apples", "Dragon fruit", "Pear(s)", "Apple(s)", "Prune(s)", "Pitted prune(s)", "Dried prunes", "Raspberry puree", "Mash mango", "Mash passion", "Pêches", "Peaches in syrup", "Dried grapes", "Rhubarb", "Lemon zests", "Orange zests"]),
    ("Nuts:", ["Amandes concassées", "Amandes en poudre vahiné", "Amandes", "Slivered almonds", "Vanilla slivered almonds", "Powdered almonds", "Roasted almonds", "Walnut kernels", "Chia seeds", "Pumpkin seeds", "Cumin seeds", "Sesame seeds", "White sesame seeds", "Golden sesame seeds", "Nutmeg", "Nutmeg powder", "Grated nutmeg", "Hazelnuts", "Shelled hazelnuts", "Roasted hazelnuts", "Chopped hazelnuts", "Hazelnut powder", "Whole hazelnuts", "Walnuts", "Crushed nuts", "Crushed nuts", "Grenoble walnut", "Nutmeg", "Nutmeg powder", "Grated nutmeg", "Nutmeg", "Powdered hazelnut pack(s)", "Pine pinions", "Pistachios", "Chopped pistachios", "Unsalted pistachios", "Hazelnut powder", "Pistachio powder", "Almond powder", "Sesame", "White sesame"])
]

main_frame = ctk.CTkFrame(app)
main_frame.pack(fill="both", expand=True, padx=10, pady=10)  # Réduisez les paddings si nécessaire

# Configurez la grille principale pour l"expansion
for i in range(3):  # Ajustez le nombre de lignes si nécessaire
    main_frame.grid_rowconfigure(i, weight=1)
    for j in range(5):  # Ajustez le nombre de colonnes si nécessaire
        main_frame.grid_columnconfigure(j, weight=1)

all_checkboxes = []
widgets_mapping = {}

for i, (category_name, items) in enumerate(item_categories):
    row = i // 5
    col = i % 5

    # Créez le CTkScrollableFrame avec un padding interne plus grand pour permettre un contenu plus large
    scrollable_frame = ctk.CTkScrollableFrame(main_frame, corner_radius=10)  # Ajustez fg_color et corner_radius selon vos besoins
    scrollable_frame.grid(row=row, column=col, sticky="nsew", padx=10, pady=10)  # Réduisez le padding externe si nécessaire

    label = ctk.CTkLabel(scrollable_frame, text=category_name, anchor="w")
    label.pack(fill="x", padx=2, pady=2)  # Réduisez le padding si nécessaire

    for item in items:
        #print(item)
        unit_info = find_table_containing_string(item_units, item)
        if unit_info == "Not find":
            print(item)

        item_frame = ctk.CTkFrame(scrollable_frame)
        item_frame.pack(fill="x", expand=True, padx=0, pady=4)  # Réduisez le padding si nécessaire

        label = ctk.CTkLabel(item_frame, text=item, width=200, height=20)  # Vous pouvez ajuster la largeur et la hauteur selon vos besoins
        label.pack(side="top", anchor="w")

        checkbox = ctk.CTkCheckBox(item_frame, text="")
        checkbox.pack(side="left", anchor="w")

        # Augmentez la largeur du combobox selon la place disponible
        #print(unit_info)
        combobox = ctk.CTkComboBox(item_frame, values=units_value[unit_info], width=350)
        combobox.pack(side="left", fill="x", expand=True, padx=1)  # Utilisez fill="x" et expand=True pour étendre

        callback = create_combobox_callback(checkbox)
        combobox.bind("<FocusIn>", callback)

        widgets_mapping[item] = (checkbox, combobox)
        #combobox.bind("<<ComboboxSelected>>", callback)
        
        all_checkboxes.append((checkbox, combobox, item))
        

# Create a frame for the buttons
buttons_frame = ctk.CTkFrame(app)
buttons_frame.pack(fill="x")

# Button to check all checkboxes
check_all_button = ctk.CTkButton(buttons_frame, text="Check all", command=lambda: set_all_checkboxes(True), corner_radius=5)
check_all_button.pack(side="left", padx=10, pady=10)

# Button to uncheck all checkboxes
uncheck_all_button = ctk.CTkButton(buttons_frame, text="Uncheck all", command=lambda: set_all_checkboxes(False), corner_radius=5)
uncheck_all_button.pack(side="left", padx=10, pady=10)

# Bouton pour cocher les articles de "Panier 1"
btn_panier_1 = ctk.CTkButton(buttons_frame, text="Cart 1", command=check_cart_1, corner_radius=5)
btn_panier_1.pack(side="left", padx=10, pady=10)

# Bouton pour cocher les articles de "Panier 2"
btn_panier_2 = ctk.CTkButton(buttons_frame, text="Cart 2", command=check_cart_2, corner_radius=5)
btn_panier_2.pack(side="left", padx=10, pady=10)

# Bouton pour cocher les articles de "Panier 3"
btn_panier_3 = ctk.CTkButton(buttons_frame, text="Cart 3", command=check_cart_3, corner_radius=5)
btn_panier_3.pack(side="left", padx=10, pady=10)

# Bouton pour appeler la fonction All()
submit_button = ctk.CTkButton(buttons_frame, text="Order", command=ouvrir_nouvelle_fenetre, corner_radius=5)
submit_button.pack(side="left", padx=15, pady=10)

comboboxe = ctk.CTkComboBox(buttons_frame, values=["1","2","3","4","5","6","7","8","9","10"], corner_radius=5)
comboboxe.set("No. people.")  # Texte d"espace réservé
comboboxe.pack(side="left", padx=10, pady=10)
comboboxe.bind("<FocusIn>", on_combobox_click)
comboboxe.bind("<FocusOut>", on_combobox_focusout)

# Start the main loop
app.mainloop()

2025-01-22 12:40:15.578 Python[16690:18613714] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-22 12:40:15.578 Python[16690:18613714] +[IMKInputSession subclass]: chose IMKInputSession_Modern


{'Strong chorizo': '0.2 kg', 'Rabbit': '2.25 kg', 'Dry white wine': '0.4 l', 'Garlic': '0.1 kg', 'Zucchini(s)': '6.0', 'Lenses': '0.5 kg', 'Onion(s)': '6', 'Red onion(s)': '6.0', 'Bell pepper(s)': '6', 'Pumpkin': '6.0', 'Soybeans': '0.1 kg', 'Tomato(s)': '6.0', 'Bay leaf(s)': '6.0', 'Rice leaves': '18.0', 'Ginger': '1 u', 'Ginger powder': '1 u', 'Paprika': '1 u', 'Pepper': '1 u', 'Black pepper': '1 u', 'Salt': '1 u', 'Thyme': '1 string(s)', 'Green curry paste': '1 u', 'Milk': '0.5 l', 'Coconut milk': '0.6 l', 'Grated Parmesan': '0.2 kg', 'Flour': '0.2 kg', 'Yeast': '1 bag(s)', 'Honey': '1 u', 'Agave syrup': '0.126 l', 'Sugar': '0.2 kg', 'Powdered sugar': '0.03 kg', 'Liquid vanilla': '0.06 l', 'White egg(s)': '6.0', 'Oat flakes': '0.18 kg', 'Oil': '1 u', 'Olive oil': '0.02 l', 'Egg(s)': '6.0', 'Quinoa': '0.3 kg', 'Rice vermicelli': '0.1 kg', 'Soy sauce': '0.1 kg', 'Tahini': '0.27 kg', 'Banana(s)': '6.0', 'Apple(s)': '6.0', 'Prune(s)': '24', 'Pitted prune(s)': '30.0', 'Raspberry puree': 

In [ ]:
# DON'T NEED TO BE COMPUTED
# TEST FULL ALGO
cart = {"Pork tenderloin": "1 kg", "Poultry": "800 g", "Lemon juice": "1 l", "Garlic": "6 pod(s)", "Red beet(s)": "8", "Carottes": "0.75 kg", "Paris mushrooms": "50 g", "Zucchini(s)": "6", "Green bean(s)": "0.75 kg", "Coral lentils": "150 g", "Green lentils": "150 g", "Turnip(s)": "6", "Onion(s)": "6", "Red bell pepper(s)": "3.0", "Ras-el-hanout": "50 g", "Shallot(s)": "3", "Parsley bouquet": "1", "Cinnamon": "100 g", "Coriander": "3 bouquet(s)", "Cumin": "100 g", "Turmeric": "50 g", "Ginger powder": "50 g", "Paprika": "50 g", "Pepper": "150 g", "Salt": "150 g", "Fish": "1.2 kg", "Pâte feuilletée": "2.0", "Pâte(s) brisée(s)": "2.0", "Whipping cream": "800 g", "Fresh cream": "1.2 kg", "Goat's cheese": "250 g", "Milk": "1.5 l", "Butter": "200 g", "Bitter cocoa powder": "180 g", "Cocoa powder": "250 g", "White chocolate": "450 g", "Flour": "800 g", "Gelatin": "6 leaf(s)", "Yeast": "2 bag(s)", "Baking powder": "6 bag(s)", "Honey": "150 g", "Sugar": "500 g", "Crystal sugar": "400 g", "Powdered sugar": "1.35 kg", "Semolina sugar": "300 g", "Vanilla sugar": "4 bag(s)", "Liquid vanilla": "6", "Soluble coffee": "100 g", "Breadcrumbs": "100 g", "Corn starch": "60 g", "Oil": "1 l", "Olive oil": "1 l", "Frying oil": "1 l", "Egg yolk(s)": "12", "Maïzena": "100 g", "Egg(s)": "24", "Old-fashioned mustard": "300 g", "Soy sauce": "50 cl", "Cherries": "1 kg", "Prune(s)": "30", "Walnut kernels": "50 g", "Sesame seeds": "100 g", "Walnuts": "100 g", "Nutmeg": "130 g"}

print(all_algo(6,cart))


Suggestion: With what you have in your basket you can make :
- a Black forest without cherries
- a Fish fingers

Suggestions of recipes for 6 person(s):
- You need 5.0 'Parsley bouquet' and 6.0  'Puff pastry' to cook a 'Filet mignon en croûte'
- You need 6.0 'Puff pastry' to cook a 'Beet and goat's cheese tart'
- You need 6.0 'Pâte brisée' to cook a 'Cherry pie in jars'
- You need 6.0 'Pumpkin' to cook a 'Pumpkin muffins'
- You need 6.0 'Pumpkin' to cook a 'Pumpkin waffle'
- You need 6.0 'Pumpkin' to cook a 'Sweet pumpkin flan'
- You need 0.252 kg of 'Pumpkin(s)' to cook a 'Pumpkin Muffins'
- You need 12.0 'Potato(s)' to cook a 'Rostis house'
- You need 0.078 kg of 'Green olives' to cook a 'Zucchini tapenade'
- You need 6.0 'Lemon(s)' to cook a 'Ciambella'
- You need 6.0 'Lemon(s)' to cook a 'Lemon soufflé'
- You need 0.102 kg of 'Dark pastry chocolate' to cook a 'Chocolate Religieuse'
- You need 0.15 kg of 'Peaches in syrup' to cook a 'Peach cakes'
- You need 6.0 'Lamb collar(s)' to c

In [ ]:
# DON'T NEED TO BE COMPUTED
# TEST SHOPPING CART

items_r = extract_items('Halva with pistachio', data_pers(6))
cart = {'Apple(s)': '6.0', 'Egg(s)': '6.0', 'Flour': '0.17 kg', 'Liquid vanilla': '0.06 l', 'Milk': '0.4 l', 'Prune(s)': '24', 'Salt': '1 u', 'Sugar': '0.2 kg', 'Yeast': '1 sachet(s)'}

#print(items_r)
cart_2  = ['Bay leaf(s)', 'Dry white wine', 'Garlic', 'Olive oil', 'Onion(s)', 'Pepper', 'Pitted prune(s)', 'Potato(s)', 'Rabbit', 'Salt', 'Thyme', 'Chicken bouillon cube(s)', 'Garlic', 'Grated Parmesan', 'Olive oil', 'Paprika', 'Pepper', 'Pumpkin', 'Salt', 'Strong chorizo', 'Carotte(s)', 'Egg(s)', 'Garlic', 'Ginger powder', 'Olive oil', 'Onion(s)', 'Red onion(s)', 'Rice leaves', 'Rice vermicelli', 'Soy sauce', 'Soybeans', 'Bell pepper(s)', 'Carotte(s)', 'Coconut milk', 'Garlic', 'Ginger', 'Green curry paste', 'Oil', 'Olive oil', 'Onion(s)', 'Quinoa', 'Salt', 'Zucchini(s)', 'Almond powder', 'Mash mango', 'Mash passion', 'Powdered sugar', 'Raspberry puree', 'Sugar', 'White egg(s)', 'Black pepper', 'Lenses', 'Olive oil', 'Salt', 'Tomato(s)', 'Apple(s)', 'Egg(s)', 'Flour', 'Liquid vanilla', 'Milk', 'Prune(s)', 'Salt', 'Sugar', 'Yeast', 'Banana(s)', 'Honey', 'Oat flakes', 'Agave syrup', 'Pistachios', 'Tahini']
qties_2 = ['6.0', '0.4 l', '0.1 kg', '0.1 l', '6', '1 u', '30.0', '12.0', '2.25 kg', '1 u', '1 string(s)', '6.0', '0.1 kg', '0.2 kg', '0.1 l', '1 u', '1 u', '6.0', '1 u', '0.2 kg', '6.0', '6.0', '0.1 kg', '1 u', '0.09 l', '6', '6.0', '18.0', '0.1 kg', '0.1 kg', '0.1 kg', '6', '6.0', '0.6 l', '0.1 kg', '1 u', '1 u', '1 u', '1 u', '6', '0.3 kg', '1 u', '6.0', '0.1 kg', '0.2 kg', '0.2 kg', '0.03 kg', '0.16 kg', '0.1 kg', '6.0', '1 u', '0.5 kg', '0.02 l', '1 u', '6.0', '6.0', '6.0', '0.2 kg', '0.06 l', '0.5 l', '24', '1 u', '0.2 kg', '1 bag(s)', '6.0', '1 u', '0.18 kg', '0.126 l', '0.09 kg', '0.27 kg']
my_dict = dict(zip(cart_2, qties_2))

#for i in range(len(cart_2)):
#    if cart_2[i] in items_r:
#        print(f"{i}: {cart_2[i]} : {qties_2[i]}")

#print(my_dict)
print(all_algo(6, my_dict))

({'Butter': '0.042 kg'}, 'Sweet pumpkin flan')
Butter
({'Butter': '0.024 kg'}, 'Rostis house')
Butter
({'Olive oil': '70.0 ml'}, 'Vegetarian rolls')
Olive oil
({'White pepper': '1 u'}, 'Vegetarian quinoa wok')
({'Butter': '0.18 kg'}, 'Four-quarter express with banana')
Butter
({'Whipped cream': '0.3 kg'}, 'Raspberry Mango Entremet')
Whipped cream
({'Olive oil': '70.0 ml'}, 'Lentils with tomatoes and onions')
Olive oil
({'Powdered sugar': '150.0 g'}, 'Coconut milk and caramel custard')
Powdered sugar
({'Semolina': '0.102 kg'}, 'Semolina cake with prunes')
Semolina
({'Cassava': '0.186 kg'}, 'Banana poé')
Cassava
({'Split peas': '0.498 kg'}, 'Fava')
Split peas
Suggestion: With what you have in your basket you can make :
- a Far breton with apples and prunes
- a Halva with pistachio
- a Banana oatmeal cookies

Suggestions of recipes for 6 person(s):
- You need 0.042 kg of 'Butter' to cook a 'Sweet pumpkin flan'
- You need 0.024 kg of 'Butter' to cook a 'Rostis house'
- You need 70.0 ml of 

In [42]:
# DON'T NEED TO BE COMPUTED

dic = {'Agave syrup': '0.126 l', 'Pistachios': '0.09 kg', 'Tahini': '0.27 kg'}

keys_list = list(dic.keys())
values_list = list(dic.values())

print(keys_list)   # Prints list of all keys
print(values_list) # Prints list of all values

['Agave syrup', 'Pistachios', 'Tahini']
['0.126 l', '0.09 kg', '0.27 kg']


In [ ]:
# DON'T NEED TO BE COMPUTED
# MISSING FOOD ITEM IN A SHOPPING CART TEST

basket = {"Butter": "0.032 kg",
 "White egg(s)": "8",
 "Coffee": "1 u",
 "Whole cream": "0.216 kg",
 "Flour": "0.272 kg",
 "Egg yolk(s)": "8",
 "Milk": "0.672 l",
 "Mascarpone": "0.272 kg",
 "Powdered sugar": "0.04 kg",
 "Powdered sugar": "0.072 kg",
 "Vanilla": "2 pod(s)",
 "Egg(s)": "8"}

print(missing_item(data_pers(9), basket, "Filet mignon en croûte"))

None


In [36]:
# DON'T NEED TO BE COMPUTED

import pandas as pd
import re
from collections import Counter

def extraire_unites(cellule):
    pattern = r"\b(kg|l|u|slices?|bunches?|bouquets?|leafs?|sprigs?|boxs?|sprigs?|pots?|bags?|pods?|bâtons?)\b"
    match = re.search(pattern, str(cellule), re.IGNORECASE)
    return match.group() if match else None

# Lire les données depuis un fichier CSV
df = pd.read_csv('Data/test3_eng.csv', sep=';')

for col in df.columns:
    # Comptabiliser les unités de mesure
    unites = df[col].dropna().apply(extraire_unites)
    comptage_unites = Counter(unites)

    # Comptabiliser les cellules contenant uniquement des chiffres et exclure 0
    valeurs_chiffres = df[col].apply(lambda x: (isinstance(x, (int, float)) and x != 0) and extraire_unites(x) is None).sum()

    # Compter le nombre de 0 dans chaque colonne
    valeurs_null = df[col].apply(lambda x: x == 0 or x == '0').sum()



    print(f"Column name: {col}, Unit count: {dict(comptage_unites)}, 'value': {valeurs_chiffres}, 'null': {valeurs_null}")


Column name: Dish, Unit count: {None: 542, 'Slice': 1}, 'value': 0, 'null': 0
Column name: Dried apricots, Unit count: {None: 542, 'u': 1}, 'value': 0, 'null': 542
Column name: "Turkey cutlet(s)", Unit count: {None: 543}, 'value': 3, 'null': 540
Column name: 0% fromage blanc, Unit count: {None: 540, 'kg': 3}, 'value': 0, 'null': 540
Column name: 15% liquid cream, Unit count: {None: 542, 'l': 1}, 'value': 0, 'null': 542
Column name: 3% light cream, Unit count: {None: 542, 'kg': 1}, 'value': 0, 'null': 542
Column name: 30% cream, Unit count: {None: 543}, 'value': 0, 'null': 543
Column name: 30% fresh cream, Unit count: {None: 541, 'kg': 2}, 'value': 0, 'null': 541
Column name: 30% full cream, Unit count: {None: 542, 'kg': 1}, 'value': 0, 'null': 542
Column name: 35% liquid cream, Unit count: {None: 542, 'u': 1}, 'value': 0, 'null': 542
Column name: 4-spice blend, Unit count: {None: 540, 'u': 2, 'kg': 1}, 'value': 0, 'null': 540
Column name: 5-berry mix, Unit count: {None: 542, 'u': 1}, '

In [29]:
# DON'T NEED TO BE COMPUTED

for unite, cols in colonnes_classees.items():
    print(f"Unit: {unite}, Column: {cols}")

Unit: unknow, Column: ['30% cream', '35% liquid cream', '5-berry mix', 'Acacia honey', 'Amaretto', 'Armagnac', 'Baguette', "Baker's yeast", 'Baking powder', 'Balsamic vinegar', 'Balsamic vinegar cream with toscoro berries', 'Basil', 'Beef', 'Bicarbonate of soda', 'Bitter almond aroma', 'Bitter cocoa powder', 'Black pepper', 'Blond sugar', 'Bolognese sauce', 'Bottle(s) brut champagne', 'Bottle(s) red wine', 'Bread', 'Brioche bread', 'Broth', 'Broth cube(s)', 'Bulot(s)', 'Butter hazelnut', 'Cabbage', 'Can(s) tomato paste', 'Caramel', 'Caviar', 'Cheddar', 'Chervil', 'Chicken', 'Chicken broth', 'Chili pepper', 'Chipolatas', 'Chocolate mousse', 'Chocolate powder', 'Chocolate vermicelli', 'Chopped coriander', 'Chopped tarragon', 'Ciboulette', 'Cider vinegar', 'Cinnamon powder', 'Clam(s)', 'Coffee', 'Cognac', 'Colombo', 'Condensed milk', 'Coriander', 'Coriander powder', 'Corned-beef', 'Country bread', 'Couscous', 'Cream pudding', 'Croutons', 'Crumb bread', 'Crêpes dentelles', 'Cumin powder', 

In [30]:
# DON'T NEED TO BE COMPUTED

item_meats = ["Chicken wing(s)", "Andouillette(s)", "Bacon", "Chicken breast(s)", "Beef", "Ground beef", "Duck(s)", "Sausage meat", "Chipolatas", "Chorizo", "Mild chorizo", "Strong chorizo", "Colliers d'agneaux", "Corned-beef", "Rabbit legs", "Chicken thighs", "Lamb chop", "Diced ham", "Beef rib steak", "Shoulder(s) of lamb", "Escalopes de dinde", "Escalopes de poulet", "Beef tenderloin", "Pork tenderloin", "Pork tenderloin", "Smoked tenderloin", "Beef tenderloin", "Poultry fillets", "Foie gras", "Poultry stock", "Veal stock", "Poultry stock", "Leg of lamb", "Graisse doie", "Guanciale", "Poultry gizzard(s)", "Ham", "Cooked ham", "Parma ham", "Veal juice", "Rabbit", "Smoked bacon", "Bacon", "Smoked bacon", "Merguez", "Beef chuck", "Clam(s)", "Pack(s) of bacon", "Package(s) smoked bacon", "Paupiettes", "Rump steak(s)", "Pigeon", "Guinea fowl", "Lamb belly", "Pork belly", "Pork", "Ground pork", "Chicken", "Leftover meat", "Rumsteak(s)", "Sausage(s)", "Sausage(s)", "Smoked sausage(s)", "Sausage", "Garlic sausage", "Saddle of lamb", "Chopped steak(s)", "Guinea fowl supreme", "Capon supremes", "Ham heel", "Slices of ham", "Slice(s) of Bayonne ham", "Parma ham slice(s)", "Meat", "Beef", "Ground beef", "Lean beef", "Minced meat", "Ground meat 5% (fat)", "Poultry", "Lamb shoulder", "Pork shoulder"]
item_drinks = ["Amaretto", "Armagnac", "Bottle(s) brut champagne", "Bottle(s) red wine", "Cognac", "Colombo", "Grand marnier", "Orange juice", "Lemon juice", "Lime juice", "Lychee juice", "Grapefruit juice", "Porto", "Rhum", "Dark rum", "Vergeoise blonde", "Vermouth", "White wine", "Dry white wine", "Madeira wine", "Yellow wine", "Rosé wine", "Red wine", "Whisky"]
item_vegetables = ["Garlic", "Dehydrated garlic", "Garlic powder", "White asparagus", "Green asparagus(s)", "Eggplant(s)", "Avocado(s)", "Tray(s) of button mushrooms", "Cherry tomato tray(s)", "Beet(s)", "Red beet(s)", "White leek(s)", "Chard(s)", "Jar of pickled bell pepper(s)", "Can(s) tomato paste", "Brocoli(s)", "Carottes", "Mushrooms", "Paris mushrooms", "Chanterelle(s)", "White cabbage", "Brussels sprouts", "Kohlrabi", "Green cabbage", "Cabbage(s)", "Chou(x) de bruxelle", "Cauliflower", "Heart of palm", "Combava", "Tomato concentrate", "Cucumber(s)", "Pickles", "Tomato coulis", "Butternut squash", "Zucchini(s)", "Porcini mushrooms", "Artichoke heart", "Endives", "Flageolet(s)", "Giraumon", "Clove(s)", "Grappes de tomates", "White beans", "Pink beans", "Kidney beans", "Green bean(s)", "Jeunes pousses d'épinard", "Lettuce", "Lenses", "Coral lentils", "Green lentils", "Frozen vegetable medley", "Cassava", "Corn", "Mini carottes", "Mousseron(s)", "Turnip(s)", "Onion(s)", "White onion(s)", "Sweet onion(s)", "Chopped onion(s)", "Yellow onion(s)", "Oignon(s) moyen(s)", "Red onion(s)", "Green onion(s)", "Chopped onion(s)", "Green olives", "Olives", "Black olives", "Pitted black olives", "Green olives", "Parsnips", "Sweet potato(s)", "Truffle beads", "Small peppers", "Small green peppers", "Small red peppers", "Small onion(s)", "Small tomatoes", "Peas", "Chili pepper", "Espelette pepper", "Cayenne pepper", "Sweet pepper", "Sweet pepper powder", "Chilli powder", "Hot pepper(s)", "Dried pepper", "Green pepper", "Red pepper(s)", "Oyster mushroom(s)", "Leek(s)", "Pears in syrup", "Split peas", "Chickpeas", "Sweet peas", "Bell pepper(s)", "Yellow bell pepper(s)", "Red bell pepper(s)", "Green bell pepper(s)", "Pumpkin", "Pumpkin(s)", "Pousses d'épinard", "Bamboo shoot", "Soybean pods", "Tomato pulp", "Mash tomato", "Ras-el-hanout", "Rocket salad", "Salad", "Shiitake(s)", "Soybeans", "Tapioca", "Tomato(s)", "Tomate(s) cerise(s)", "Tomate(s) confite(s)", "Tomate(s) pelée(s)", "Dried tomato(s)", "Chopped tomatoes", "Medium tomatoes", "Dried tomatoes", "Jerusalem artichoke", "Black truffle(s)", "Cabbage", "Shallot(s)", "Chopped shallot(s)", "Minced shallot(s)", "Chopped spinach", "Spinach(s)"]
item_herbspices = ["Dill", "Chopped dill", "Basil", "Parsley bouquet", "Garnished bouquet(s)", "Celery stalk(s)", "Fennel bulb", "Bâtons citronnelle", "Cinnamon", "Cinnamon powder", "Chervil", "Chives", "Clove(s)", "Coriander", "Chopped coriander", "Coriander powder", "Cumin", "Cumin powder", "Turmeric", "Curry", "Celery root", "Spices", "Chopped tarragon", "Fennel", "Bay leaf(s)", "Filo sheets", "Rice leaves", "Herbs", "Edible flowers", "Salt flowers", "Trim", "Ginger", "Ginger powder", "Fennel seeds", "Mustard seeds", "Coarse salt", "Herbs", "Provence herbs", "Dried herbs", "Kubor", "Laurel", "Menthe", "Fresh mint", "4-spice blend", "5-berry mix", "Oregano", "Paprika", "Smoked paprika", "Parsley", "Chopped parsley", "Curled parsley", "Chopped parsley", "Chopped parsley", "Flat-leaf parsley", "Pepper", "White pepper", "Lebanese pepper", "Black pepper", "Yellow curry powder", "Rosemary", "Safran", "Buckwheat", "Savory", "Sauge", "Salt", "Garlic and parsley tablet(s)", "Thyme", "Rhubarb stems", "Wasabi"]
item_fishes = ["Bisques de homard", "Bulot(s)", "Bâtons surimis", "Bâtonnets surimis", "Caviar", "Crab(s)", "Shrimp(s)", "Grey shrimps", "Pink shrimps", "Back of fresh salmon", "Cuttlefish ink", "Swordfish", "Tarragon", "Tournedos filet", "Salmon fillet(s)", "Pike-perch fillets", "Sole fillets", "Fish stock", "Gambas", "Lambis", "Lobsters", "Langoustines", "Mackerel(s)", "Morel(s)", "Dried morel(s)", "Salt cod", "Mussels", "Scallops", "Nuoc mâm", "Cod eggs", "Salmon steak(s)", "Fish", "Mutti garlic polpa", "Octopus(s)", "Quenelle(s)", "Monkfish tail(s)", "Scallops", "Pike-perch", "Sardine", "Salmon", "Cuttlefish", "Soup", "Tuna steak(s)", "Tuna", "Slices of smoked salmon", "Smoked trout", "Trout(s)"]
item_pates = ["Peanut paste", "Green curry paste", "Hazelnut paste", "Pistachio paste", "Rice paste", "Pâte feuilletée", "Filo dough", "Shortbread dough", "Pizza dough", "Raviole dough", "Pie dough", "Pâte(s) brisée(s)", "Pâte feuilletée", "Pasta", "Pâte feuilletée"]
item_milk = ["Boursin(s)", "Blue Bresse", "Brick(s) of liquid crème fraîche", "Burrata", "Goat's cheese log", "Bûches chèvre", "Camembert", "Cancoillotte", "Fresh square", "Chantilly", "Cheddar", "Goat cheese", "Comté cheese", "Shredded Comté cheese", "Parmesan shavings", "Crème", "Light cream", "Custard cream", "Whipped cream", "Coconut cream", "Chestnut cream", "Balsamic vinegar cream with toscoro berries", "Whole cream", "30% full cream", "Whipping cream", "Crème fleurette 30%", "Whipped cream", "Heavy cream", "Fresh cream", "30% fresh cream", "30% fresh cream", "3% light cream", "Whole cream", "Liquid fresh cream", "Whole cream", "Semi-thick fresh cream", "Heavy cream", "15% liquid cream", "35% liquid cream", "30% cream", "Light cream", "Mounted cream", "Cream pudding", "Sour cream", "Thick cream", "Emmental", "Grated Emmental cheese", "Fiscella", "Feta cheese", "Cheese", "Garlic and herb cheese", "Cottage cheese", "0% fromage blanc", "Goat's cheese", "Feta cheese", "Grated cheese", "Chocolate ice cream", "Vanilla ice cream", "Gorgonzola", "Gruyère", "Grated Gruyère", "Philadelphia", "Milk", "Condensed milk", "Unsweetened condensed milk", "Sweetened condensed milk", "Almond milk", "Goat's milk", "Coconut milk", "Soy milk", "Semi-skimmed milk", "Whole milk", "Plant milk", "Skim milk", "Manicotti", "Maroilles", "Mascarpone", "Mozzarella", "Grated mozzarella", "Parmesan", "Grated Parmesan", "Pecorino", "Philadelphia", "Pizzoccheri", "Raifort", "Reblochon", "Ricotta", "Goat's cheese log", "Roquefort", "Philadelphia", "Sainte-maure de touraine", "Yogurt"]
item_bakery = ["Lemon vanilla flavor", "Bitter almond aroma", "Vanilla flavor", "Butter", "Peanut butter", "Semi-salted butter", "Soft butter", "Salted butter", "Soft butter", "Bicarbonate of soda", "Biscuits", "Spoon cookies", "Bitter cocoa", "Bitter cocoa powder", "Cocoa powder", "Unsweetened cocoa powder", "Caramel", "Salted butter caramel", "Carrés chocolat noir", "Brown sugar", "Chocolate", "Bitter chocolate powder", "Milk chocolate", "White chocolate", "Chocolate powder", "Dark chocolate", "50% dark chocolate", "Dark pastry chocolate", "70% cocoa dark chocolate", "Dark baking chocolate", "Pastry chocolate", "Food coloring red", "Chocolate chips", "Orange blossom water", "Vanilla essence", "Vanilla extract", "Flour", "Buckwheat flour", "Wheat flour", "Rice flour", "Sticky rice flour", "Sticky rice flour", "Flour t55", "Sifted flour", "Gelatin sheets", "Orange blossom", "Yellow food coloring drops", "Red food coloring drop(s)", "Gelatin", "Levure", "Levure boulangère", "Levure chimique", "Margarine", "Margarine fondue", "Margarine végétale", "Miel", "Acacia honey", "Miel à la truffe", "Mini snickers", "Morceaux sucre", "Nesquik", "Noisette de beurre", "Nutella", "Oreos", "Paquet(s) boudoirs", "Paquet(s) gaufrette", "Petits beurres", "Plaques pralinoise", "Almond powder", "Poudre de cacao", "Pralinoise", "Chocolate chips", "Dark chocolate chips", "Sablés bretons", "Agave syrup", "Maple syrup", "Smen", "Spéculoos", "Sucre", "Sucre blond", "Sucre brun", "Sucre complet", "Sucre cristal", "Sucre de canne", "Sucre en poudre", "Sucre glace", "Sucre grain", "Sucre roux", "Sucre semoule", "Sucre vanillé", "Tablette(s) chocolat au lait", "Tablette(s) chocolat blanc", "Tablette(s) chocolat noir", "Twix", "Vanille", "Vanille en poudre", "Vanille liquide", "Vermicelle de chocolat", "Édulcorant"]
item_others = ["White egg(s)", "Broth", "Vegetable broth cube", "Beef broth", "Vegetable broth", "Chicken broth", "Poultry broth", "Peanuts", "Coffee", "Soluble coffee", "Breadcrumbs", "Coques", "Corn flakes", "Crackers salés", "Croûtons", "Crêpes dentelles", "Crêpes", "Sachet(s) de crêpes dentelles", "Cubes bouillon", "Cubes bouillon de bœuf", "Cubes bouillon de légumes", "Cubes bouillon de viande", "Cubes bouillon de volaille", "Cure-dents", "Darne(s) vivaneau", "Encornet(s)", "Brick leaf(s)", "Lasagne leaf(s)", "Oat flakes", "Fond de tarte", "Fraise(s) tagada(s)", "Fécule", "Fécule de maïs", "Galettes", "Galettes de riz", "Garni", "Gavotte", "Gelée instantanée", "Granny", "Huile", "Peanut oil", "Olive oil", "Huile de coco", "Huile de colza", "Huile de friture", "Huile de noix", "Huile de sésame", "Huile de tournesol", "Huile végétale", "Egg yolk(s)", "Macaronis", "Maizena", "Maïzena", "Mc vitties", "Mie de pain", "Mirin", "Mousse au chocolat", "Pain", "Pain au lait", "Pain brioché", "Gingerbread", "Pain de campagne", "Pain de mie", "Pain rassis", "Pain(s) à hamburger(s)", "Paquet(s) chamallow", "Perles du japon", "Oat bran", "Tortillas de maïs", "Tortillas", "Vinaigre", "Vinaigre balsamique", "Vinaigre balsamique blanc", "Vinaigre blanc", "Spirit vinegar", "Vinaigre de cidre", "Vinaigre de riz", "Vinaigre de vin rouge", "Vinaigre de xérès", "Vol-au-vent", "Wraps", "Œuf(s)"]
item_sides = ["Baguette", "Belles slices de pain de campagne", "Brioche", "Conchiglionis", "Couscous", "Gnocchis", "Lasagnes", "Linguine(s) crue(s)", "Nouilles", "Pennes", "Pomme de terre bintje", "Pomme(s) de terre", "Pommes de terre", "Quinoa", "Rigatonis", "Riz", "Riz arborio", "Riz gluant", "Riz long", "Riz rond", "Riz soufflé nature", "Riz thaï", "Riz à risotto", "Semoule", "Semoule fine", "Spaghettis", "Tagliatelles", "Tortellinis", "Vermicelle chinois", "Vermicelle de riz"]
item_sauces = ["Gouttes de tabasco", "Harissa", "Ketchup", "Mayonnaise", "Moutarde", "Moutarde forte", "Old-fashioned mustard", "Sauce bolognaise", "Sauce chocolat", "Sauce nuoc mam", "Sauce soja", "Sauce teriyaki", "Sauce tomate", "Sauce yakitori kikkoman", "Tabasco", "Tahini"]
item_fruits = ["Abricots secs", "Agar-agar", "Ananas", "Ananas victoria", "Baies roses", "Baies", "Banane(s)", "Belles pommes", "Cardamome", "Cerises", "Citrons", "Citrons verts", "Coing", "Compote de pomme", "Confiture de fraises", "Confiture de framboise", "Coulis de framboise", "Coulis de fruit rouge", "Cranberries séchées", "Cranberrys", "Câpres", "Dattes", "Dattes medjool", "Dattes séchées", "Etoile de badiane", "Figues", "Figues blanche(s)", "Figues séchées", "Fraise(s)", "Framboise(s)", "Fruit(s) de la passion", "Fruit(s) rouge(s)", "Fruits rouges", "Kakis", "Kiwi(s)", "Litchi au sirop", "Litchi(s)", "Mangue(s)", "Mirabelles", "Myrtilles", "Noix de coco râpée", "Oranges", "Pamplemousse", "Papaye", "Petites pommes", "Fruit du dragon", "Poire(s)", "Pomme(s)", "Pruneaux", "Pruneaux dénoyautés", "Pruneaux secs", "Purée de framboises", "Purée de mangue", "Purée de passion", "Peach(es)", "Peaches au sirop", "Raisins secs", "Rhubarbe", "Zestes de citron", "Zestes d'oranges"]
item_nuts = ["Amandes concassées", "Amandes en poudre vahiné", "Amandes", "Amandes effilées", "Amandes effilées vahiné", "Amandes en poudre", "Amandes grillées", "Cerneaux de noix", "Châtaignes", "Châtaignes en bocal", "Graines de chia", "Graines de courge", "Graines de cumin", "Graines de sésame", "Graines de sésame blanc", "Graines de sésame doré", "Muscade", "Muscade en poudre", "Muscade râpé", "Noisettes", "Noisettes décortiquées", "Noisettes grillées", "Noisettes concassées", "Noisettes en poudre", "Noisettes entières", "Noix", "Noix concassées", "Noix concassées", "Noix de grenoble", "Noix de muscade", "Noix de muscade en poudre", "Noix de muscade râpée", "Noix muscade", "Paquet(s) noisette en poudre(s)", "Pignons de pin", "Pistaches", "Pistaches hachées", "Pistaches non salées", "Poudre de noisette", "Poudre de pistache", "Poudre d’amandes", "Sésame", "Sésame blanc"]

item = item_meats
item += item_drinks + item_vegetables + item_herbspices + item_fishes + item_pates + item_milk + item_bakery + item_others + item_sides + item_sauces + item_fruits + item_nuts
print(len(item))

788


In [31]:
# DON'T NEED TO BE COMPUTED

import pandas as pd
items = pd.read_csv('Data/Item_eng.csv',sep=',')
ol_alim = items['Item'].to_list()
def trouver_elements_non_communs(liste1, liste2):
    non_communs = []

    # Vérifier les éléments de la première liste
    for element in liste1:
        if element not in liste2:
            non_communs.append(element)

    # Vérifier les éléments de la deuxième liste
    for element in liste2:
        if element not in liste1:
            non_communs.append(element)

    return non_communs

alim_non_ajouté = trouver_elements_non_communs(item, ol_alim)
len(ol_alim)

790

In [1]:
# DON'T NEED TO BE COMPUTED

# DON'T RUN THIS CELL
# DON'T RUN THIS CELL
# DON'T RUN THIS CELL
# DON'T RUN THIS CELL
# DON'T RUN THIS CELL

alim1_meats = []
alim1_drinks = []
alim1_vegetabkes = []
alim1_herbspices = []
alim1_fishes = []
alim1_pates = []
alim1_milk = [] 
alim1_bakery = []
alim1_others = []
alim1_sides = []
alim1_sauces = []
alim1_fruits = []
alim1_nuts = []

'''for i in range(len(ol_alim)):
    x = input(f'{i+1}/790 : {ol_alim[i]}')
    print(f"Vous avez entré le nombre entier : {x}")

    if x == '0':
        continue
    if x == '1':
        alim1_viandes.append(ol_alim[i])
    if x == '2':
        alim1_boissons.append(ol_alim[i])
    if x == '3':
        alim1_legumes.append(ol_alim[i])
    if x == '4':
        alim1_herepices.append(ol_alim[i])
    if x == '5':
        alim1_poissons.append(ol_alim[i])
    if x == '6':
        alim1_pates.append(ol_alim[i])
    if x == '7':
        alim1_fromagerie.append(ol_alim[i])
    if x == '8':
        alim1_patisserie.append(ol_alim[i])
    if x == '9':
        alim1_autres.append(ol_alim[i])
    if x == '10':
        alim1_accompagnements.append(ol_alim[i])
    if x == '11':
        alim1_sauces.append(ol_alim[i])
    if x == '12':
        alim1_fruits.append(ol_alim[i])
    if x == '13':
        alim1_noix.append(ol_alim[i])'''

NameError: name 'ol_alim' is not defined

In [32]:
# DON'T NEED TO BE COMPUTED

item_meats = ["Chicken wing(s)", "Andouillette(s)", "Bacon", "Chicken breast(s)", "Beef", "Ground beef", "Duck(s)", "Sausage meat", "Chipolatas", "Chorizo", "Mild chorizo", "Strong chorizo", "Colliers d'agneaux", "Corned-beef", "Rabbit legs", "Chicken thighs", "Lamb chop", "Diced ham", "Beef rib steak", "Shoulder(s) of lamb", "Escalopes de dinde", "Escalopes de poulet", "Beef tenderloin", "Pork tenderloin", "Pork tenderloin", "Smoked tenderloin", "Beef tenderloin", "Poultry fillets", "Foie gras", "Poultry stock", "Veal stock", "Poultry stock", "Gigot dagneau", "Graisse doie", "Guanciale", "Poultry gizzard(s)", "Ham", "Cooked ham", "Parma ham", "Veal juice", "Rabbit", "Smoked bacon", "Bacon", "Smoked bacon", "Merguez", "Beef chuck", "Clam(s)", "Pack(s) of bacon", "Package(s) smoked bacon", "Paupiettes", "Rump steak(s)", "Pigeon", "Guinea fowl", "Lamb belly", "Pork belly", "Pork", "Ground pork", "Chicken", "Leftover meat", "Rumsteak(s)", "Sausage(s)", "Sausage(s)", "Smoked sausage(s)", "Sausage", "Garlic sausage", "Saddle of lamb", "Chopped steak(s)", "Guinea fowl supreme", "Capon supremes", "Ham heel", "Slices of ham", "Slice(s) of Bayonne ham", "Parma ham slice(s)", "Meat", "Beef", "Ground beef", "Lean beef", "Minced meat", "Ground meat 5% (fat)", "Poultry", "Lamb shoulder", "Pork shoulder"]
item_drinks = ["Amaretto", "Armagnac", "Bottle(s) brut champagne", "Bottle(s) red wine", "Cognac", "Colombo", "Grand marnier", "Orange juice", "Lemon juice", "Lime juice", "Lychee juice", "Grapefruit juice", "Porto", "Rhum", "Dark rum", "Vergeoise blonde", "Vermouth", "White wine", "Dry white wine", "Madeira wine", "Yellow wine", "Rosé wine", "Red wine", "Whisky"]
item_vegetables = ["Garlic", "Dehydrated garlic", "Garlic powder", "White asparagus", "Green asparagus(s)", "Eggplant(s)", "Avocado(s)", "Tray(s) of button mushrooms", "Cherry tomato tray(s)", "Beet(s)", "Red beet(s)", "White leek(s)", "Chard(s)", "Jar of pickled bell pepper(s)", "Can(s) tomato paste", "Brocoli(s)", "Carottes", "Mushrooms", "Paris mushrooms", "Chanterelle(s)", "White cabbage", "Brussels sprouts", "Kohlrabi", "Green cabbage", "Cabbage(s)", "Chou(x) de bruxelle", "Cauliflower", "Heart of palm", "Combava", "Tomato concentrate", "Cucumber(s)", "Pickles", "Tomato coulis", "Butternut squash", "Zucchini(s)", "Porcini mushrooms", "Artichoke heart", "Endives", "Flageolet(s)", "Giraumon", "Clove(s)", "Grappes de tomates", "White beans", "Pink beans", "Kidney beans", "Green bean(s)", "Jeunes pousses d'épinard", "Lettuce", "Lenses", "Coral lentils", "Green lentils", "Frozen vegetable medley", "Cassava", "Corn", "Mini carottes", "Mousseron(s)", "Turnip(s)", "Onion(s)", "White onion(s)", "Sweet onion(s)", "Chopped onion(s)", "Yellow onion(s)", "Oignon(s) moyen(s)", "Red onion(s)", "Green onion(s)", "Chopped onion(s)", "Green olives", "Olives", "Black olives", "Pitted black olives", "Green olives", "Parsnips", "Sweet potato(s)", "Truffle beads", "Small peppers", "Small green peppers", "Small red peppers", "Small onion(s)", "Small tomatoes", "Peas", "Chili pepper", "Espelette pepper", "Cayenne pepper", "Sweet pepper", "Sweet pepper powder", "Chilli powder", "Hot pepper(s)", "Dried pepper", "Green pepper", "Red pepper(s)", "Oyster mushroom(s)", "Leek(s)", "Pears in syrup", "Split peas", "Chickpeas", "Sweet peas", "Bell pepper(s)", "Yellow bell pepper(s)", "Red bell pepper(s)", "Green bell pepper(s)", "Pumpkin", "Pumpkin(s)", "Pousses d'épinard", "Bamboo shoot", "Soybean pods", "Tomato pulp", "Mash tomato", "Ras-el-hanout", "Rocket salad", "Salad", "Shiitake(s)", "Soybeans", "Tapioca", "Tomato(s)", "Tomate(s) cerise(s)", "Tomate(s) confite(s)", "Tomate(s) pelée(s)", "Dried tomato(s)", "Chopped tomatoes", "Medium tomatoes", "Dried tomatoes", "Jerusalem artichoke", "Black truffle(s)", "Cabbage", "Shallot(s)", "Chopped shallot(s)", "Minced shallot(s)", "Chopped spinach", "Spinach(s)"]
item_herbspices = ["Dill", "Chopped dill", "Basil", "Parsley bouquet", "Garnished bouquet(s)", "Celery stalk(s)", "Fennel bulb", "Bâtons citronnelle", "Cinnamon", "Cinnamon powder", "Chervil", "Chives", "Clove(s)", "Coriander", "Chopped coriander", "Coriander powder", "Cumin", "Cumin powder", "Turmeric", "Curry", "Celery root", "Spices", "Chopped tarragon", "Fennel", "Bay leaf(s)", "Filo sheets", "Rice leaves", "Herbs", "Edible flowers", "Salt flowers", "Trim", "Ginger", "Ginger powder", "Fennel seeds", "Mustard seeds", "Coarse salt", "Herbs", "Provence herbs", "Dried herbs", "Kubor", "Laurel", "Menthe", "Fresh mint", "4-spice blend", "5-berry mix", "Oregano", "Paprika", "Smoked paprika", "Parsley", "Chopped parsley", "Curled parsley", "Chopped parsley", "Chopped parsley", "Flat-leaf parsley", "Pepper", "White pepper", "Lebanese pepper", "Black pepper", "Yellow curry powder", "Rosemary", "Safran", "Buckwheat", "Savory", "Sauge", "Salt", "Garlic and parsley tablet(s)", "Thyme", "Rhubarb stems", "Wasabi"]
item_fishes = ["Bisques de homard", "Bulot(s)", "Bâtons surimis", "Bâtonnets surimis", "Caviar", "Crab(s)", "Shrimp(s)", "Grey shrimps", "Pink shrimps", "Back of fresh salmon", "Cuttlefish ink", "Swordfish", "Tarragon", "Tournedos filet", "Salmon fillet(s)", "Pike-perch fillets", "Sole fillets", "Fish stock", "Gambas", "Lambis", "Lobsters", "Langoustines", "Mackerel(s)", "Morel(s)", "Dried morel(s)", "Salt cod", "Mussels", "Scallops", "Nuoc mâm", "Cod eggs", "Salmon steak(s)", "Fish", "Mutti garlic polpa", "Octopus(s)", "Quenelle(s)", "Monkfish tail(s)", "Scallops", "Pike-perch", "Sardine", "Salmon", "Cuttlefish", "Soup", "Tuna steak(s)", "Tuna", "Slices of smoked salmon", "Smoked trout", "Trout(s)"]
item_pates = ["Peanut paste", "Green curry paste", "Hazelnut paste", "Pistachio paste", "Rice paste", "Pâte feuilletée", "Filo dough", "Shortbread dough", "Pizza dough", "Raviole dough", "Pie dough", "Pâte(s) brisée(s)", "Pâte feuilletée", "Pasta", "Pâte feuilletée"]
item_milk = ["Boursin(s)", "Blue Bresse", "Brick(s) of liquid crème fraîche", "Burrata", "Goat's cheese log", "Bûches chèvre", "Camembert", "Cancoillotte", "Fresh square", "Chantilly", "Cheddar", "Goat cheese", "Comté cheese", "Shredded Comté cheese", "Parmesan shavings", "Crème", "Light cream", "Custard cream", "Whipped cream", "Coconut cream", "Chestnut cream", "Balsamic vinegar cream with toscoro berries", "Whole cream", "30% full cream", "Whipping cream", "Crème fleurette 30%", "Whipped cream", "Heavy cream", "Fresh cream", "30% fresh cream", "30% fresh cream", "3% light cream", "Whole cream", "Liquid fresh cream", "Whole cream", "Semi-thick fresh cream", "Heavy cream", "15% liquid cream", "35% liquid cream", "30% cream", "Light cream", "Mounted cream", "Cream pudding", "Sour cream", "Thick cream", "Emmental", "Grated Emmental cheese", "Fiscella", "Feta cheese", "Cheese", "Garlic and herb cheese", "Cottage cheese", "0% fromage blanc", "Goat's cheese", "Feta cheese", "Grated cheese", "Chocolate ice cream", "Vanilla ice cream", "Gorgonzola", "Gruyère", "Grated Gruyère", "Philadelphia", "Milk", "Condensed milk", "Unsweetened condensed milk", "Sweetened condensed milk", "Almond milk", "Goat's milk", "Coconut milk", "Soy milk", "Semi-skimmed milk", "Whole milk", "Plant milk", "Skim milk", "Manicotti", "Maroilles", "Mascarpone", "Mozzarella", "Grated mozzarella", "Parmesan", "Grated Parmesan", "Pecorino", "Philadelphia", "Pizzoccheri", "Raifort", "Reblochon", "Ricotta", "Goat's cheese log", "Roquefort", "Philadelphia", "Sainte-maure de touraine", "Yogurt"]
item_bakery = ["Lemon vanilla flavor", "Bitter almond aroma", "Vanilla flavor", "Butter", "Peanut butter", "Semi-salted butter", "Soft butter", "Salted butter", "Soft butter", "Bicarbonate of soda", "Biscuits", "Spoon cookies", "Bitter cocoa", "Bitter cocoa powder", "Cocoa powder", "Unsweetened cocoa powder", "Caramel", "Salted butter caramel", "Carrés chocolat noir", "Brown sugar", "Chocolate", "Bitter chocolate powder", "Milk chocolate", "White chocolate", "Chocolate powder", "Dark chocolate", "50% dark chocolate", "Dark pastry chocolate", "70% cocoa dark chocolate", "Dark baking chocolate", "Pastry chocolate", "Food coloring red", "Chocolate chips", "Orange blossom water", "Vanilla essence", "Vanilla extract", "Flour", "Buckwheat flour", "Wheat flour", "Rice flour", "Sticky rice flour", "Sticky rice flour", "Flour t55", "Sifted flour", "Gelatin sheets", "Orange blossom", "Yellow food coloring drops", "Red food coloring drop(s)", "Gelatin", "Yeast", "Baker's yeast", "Baking powder", "Margarine", "Melted margarine", "Vegetable margarine", "Honey", "Acacia honey", "Truffle honey", "Mini snickers", "Pieces of sugar", "Nesquik", "Butter hazelnut", "Nutella", "Oreos", "Boudoir package(s)", "Wafer pack(s)", "Petits beurres", "Praline plate(s)", "Almond powder", "Cocoa powder", "Pralinoise", "Pépites de chocolat", "Pépites de chocolat noir", "Brittany shortbreads", "Fine semolina", "Maple syrup", "Smen", "Spéculoos", "Sugar", "Blond sugar", "Brown sugar", "Complete sugar", "Crystal sugar", "Cane sugar", "Powdered sugar", "Powdered sugar", "Grainy sugar", "Brown sugar", "Semolina sugar", "Vanilla sugar", "Milk chocolate tablet(s)", "White chocolate tablet(s)", "Black chocolate tablet(s)", "Twix", "Vanilla", "Vanilla powder", "Liquid vanilla", "Chocolate vermicelli", "Sweetener"]
item_others = ["White egg(s)", "Broth", "Vegetable broth cube", "Beef broth", "Vegetable broth", "Chicken broth", "Poultry broth", "Cacahuètes", "Coffee", "Soluble coffee", "Breadcrumbs", "Hulls", "Corn flakes", "Crackers salés", "Croutons", "Crêpes dentelles", "Crêpes", "Sachet(s) de crêpes dentelles", "Broth cube(s)", "Beef bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Toothpicks", "Squid(s)", "Brick pastry", "Lasagne sheet(s)", "Oat flakes", "Pie crust", "Tagada strawberry(s)", "Starch", "Corn starch", "Patties", "Rice cakes", "Gavotte", "Instant jelly", "Granny", "Oil", "Peanut oil", "Olive oil", "Coconut oil", "Rapeseed oil", "Frying oil", "Walnut oil", "Sesame oil", "Sunflower oil", "Vegetable oil", "Egg yolk(s)", "Macaronis", "Maïzena", "Maïzena", "McVitties", "Breadcrumbs", "Mirin", "Chocolate mousse", "Bread", "Milk bread", "Brioche bread", "Gingerbread", "Country bread", "Crumb bread", "Stale bread", "Hamburger bun(s)", "Marshmallow pack(s)", "Japanese pearls", "Oat bran", "Corn tortilla", "Tortillas", "Vinegar", "Balsamic vinegar", "White balsamic vinegar", "White vinegar", "Spirit vinegar", "Cider vinegar", "Rice vinegar", "Red wine vinegar", "Sherry vinegar", "Wraps", "Egg(s)"]
item_sides = ["Baguette", "Thick slices of farmhouse bread", "Brioche", "Conchiglionis", "Couscous", "Gnocchis", "Lasagnes", "Raw linguine", "Noodles", "Pennes", "Bintje potato", "Potato(es)", "Potatoes", "Quinoa", "Rigatonis", "Rice", "Arborio rice", "Sticky rice", "Long rice", "Round rice", "Plain puffed rice", "Thaï rice", "Risotto rice", "Semolina", "Fine semolina", "Spaghettis", "Tagliatelles", "Tortellinis", "Chinese vermicelli", "Rice vermicelli"]
item_sauces = ["Gouttes de tabasco", "Harissa", "Ketchup", "Mayonnaise", "Mustard", "Strong mustard", "Old-fashioned mustard", "Bolognese sauce", "Chocolate sauce", "Nuoc mam sauce", "Soy sauce", "Teriyaki sauce", "Tomato sauce", "Yakitori sauce", "Tabasco", "Tahini"]
item_fruits = ["Dried apricots", "Agar-agar", "Pineapple", "Victoria pineapple", "Baies roses", "Baies", "Banana(s)", "Nice apples", "Cardamom", "Cherries", "Lemon(s)", "Lime(s)", "Quince", "Apple compote", "Strawberry jam", "Raspberry jam", "Raspberry sauce", "Red fruit coulis", "Dried cranberries", "Cranberrys", "Capers", "Dates", "Medjool date", "Dried dates", "Star anise", "Figs", "White figs", "Dried figs", "Strawberry(s)", "Raspberry(s)", "Passion fruit(s)", "Red fruit", "Red fruit", "Khakies", "Kiwi(s)", "Litchi in syrup", "Litchi(s)", "Mango(s)", "Mirabelles", "Myrtilles", "Grated coconut", "Orange(s)", "Grapefruit", "Papaya", "Small apples", "Dragon fruit", "Pear(s)", "Bintje potato", "Prune(s)", "Pitted prune(s)", "Dried prunes", "Raspberry puree", "Mash mango", "Mash passion", "Pêches", "Peaches in syrup", "Dried grapes", "Rhubarb", "Lemon zests", "Orange zests"]
item_nuts = ["Amandes concassées", "Amandes en poudre vahiné", "Amandes", "Slivered almonds", "Vanilla slivered almonds", "Powdered almonds", "Roasted almonds", "Walnut kernels", "Chia seeds", "Pumpkin seeds", "Cumin seeds", "Sesame seeds", "White sesame seeds", "Golden sesame seeds", "Nutmeg", "Nutmeg powder", "Grated nutmeg", "Hazelnuts", "Shelled hazelnuts", "Roasted hazelnuts", "Chopped hazelnuts", "Hazelnut powder", "Whole hazelnuts", "Walnuts", "Crushed nuts", "Crushed nuts", "Grenoble walnut", "Nutmeg", "Nutmeg powder", "Grated nutmeg", "Nutmeg", "Powdered hazelnut pack(s)", "Pine pinions", "Pistachios", "Chopped pistachios", "Unsalted pistachios", "Hazelnut powder", "Pistachio powder", "Almond powder", "Sesame", "White sesame"]

In [33]:
# DON'T NEED TO BE COMPUTED

unknow = colonnes_classees["unknow"]
kg = colonnes_classees["kg"]
l = colonnes_classees["l"]
value = colonnes_classees["value"]
bunch = colonnes_classees["bunch"]
'''for i in range(len(unknow)):
    x = input(f'{unknow[i]} Voici les groupes : 1 -> kg, 2 -> l, 3 -> Value, 4 -> botte, 0 -> r ')
    print(f"Vous avez entré le nombre entier : {x}")

    if x == '0':
        continue
    if x == '1':
        kg.append(unknow[i])
    if x == '2':
        l.append(unknow[i])
    if x == '3':
        value.append(unknow[i])
    if x == '4':
        botte.append(unknow[i])'''

'for i in range(len(unknow)):\n    x = input(f\'{unknow[i]} Voici les groupes : 1 -> kg, 2 -> l, 3 -> Value, 4 -> botte, 0 -> r \')\n    print(f"Vous avez entré le nombre entier : {x}")\n\n    if x == \'0\':\n        continue\n    if x == \'1\':\n        kg.append(unknow[i])\n    if x == \'2\':\n        l.append(unknow[i])\n    if x == \'3\':\n        value.append(unknow[i])\n    if x == \'4\':\n        botte.append(unknow[i])'

In [34]:
# DON'T NEED TO BE COMPUTED

kg = colonnes_classees["kg"]
l = colonnes_classees["l"]
value = colonnes_classees["value"]
bunch = colonnes_classees["bunch"]
slice = colonnes_classees["slice"]
bunch1 = colonnes_classees["bunch"]
leaf = colonnes_classees["leaf"]
sprig = colonnes_classees["sprig"]
box = colonnes_classees["box"]
sprig1 = colonnes_classees["sprig1"]
bag = colonnes_classees["bag"]
pod = colonnes_classees["pod"]
print(len(kg), len(l))
units = kg + l + value + bunch + slice + bunch1 + leaf + sprig + box + sprig + bag + pod
print(len(units))

334 38
531


In [ ]:
# DON'T NEED TO BE COMPUTED

kg = ["Dried apricots", "Agar-agar", "Amandes concassées", "Amandes en poudre vahiné", "Amandes", "Slivered almonds", "Vanilla slivered almonds", "Powdered almonds", "Pineapple", "Chopped dill", "Lemon vanilla flavor", "White asparagus", "Beet(s)", "Red beet(s)", "Butter", "Peanut butter", "Semi-salted butter", "Soft butter", "Salted butter", "Soft butter", "Biscuits", "Bisques de homard", "Chicken breast(s)", "Chard(s)", "Boursin(s)", "Blue Bresse", "Brioche", "Burrata", "Beef", "Ground beef", "Cacahuètes", "Bitter cocoa", "Cocoa powder", "Unsweetened cocoa powder", "Soluble coffee", "Duck(s)", "Cancoillotte", "Cinnamon", "Brown sugar", "Cherries", "Walnut kernels", "Sausage meat", "Champignon(s)", "Champignon(s) de paris", "Mushrooms", "Paris mushrooms", "Chanterelle(s)", "Chantilly", "Breadcrumbs", "Chocolate", "Bitter chocolate powder", "Milk chocolate", "White chocolate", "Dark chocolate", "50% dark chocolate", "Dark pastry chocolate", "70% cocoa dark chocolate", "Dark baking chocolate", "Pastry chocolate", "Chorizo", "Strong chorizo", "Brussels sprouts", "Green cabbage", "Cabbage(s)", "Chou(x) de bruxelle", "Cauliflower", "Chestnut(s)", "Goat cheese", "Heart of palm", "Quince", "Lamb collar", "Apple compote", "Comté cheese", "Shredded Comté cheese", "Tomato concentrate", "Conchiglionis", "Raspberry jam", "Chocolate chips", "Parmesan shavings", "Hulls", "Corn flakes", "Raspberry sauce", "Red fruit coulis", "Tomato coulis", "Butternut squash", "Couscous", "Crab(s)", "Crackers salés", "Dried cranberries", "Cranberrys", "Shrimp(s)", "Shrimp(s)", "Grey shrimps", "Pink shrimps", "Crème", "Light cream", "Custard cream", "Chestnut cream", "Whole cream", "30% full cream", "Whipping cream", "Crème fleurette 30%", "Whipped cream", "Heavy cream", "Fresh cream", "30% fresh cream", "30% fresh cream", "3% light cream", "Whole cream", "Mounted cream", "Sour cream", "Crêpe dentelle(s)", "Cumin", "Turmeric", "Curry", "Capers", "Porcini mushrooms", "Celery root", "Medjool date", "Dates", "Medjool date", "Dried dates", "Diced ham", "Emmental", "Grated Emmental cheese", "Squid(s)", "Fiscella", "Flour", "Buckwheat flour", "Wheat flour", "Rice flour", "Sticky rice flour", "Sticky rice flour", "Flour t55", "Sifted flour", "Feta cheese", "Figs", "Dried figs", "Beef tenderloin", "Pork tenderloin", "Pork tenderloin", "Salmon fillet(s)", "Herbs", "Oat flakes", "Foie gras", "Strawberry(s)", "Tagada strawberry(s)", "Raspberry(s)", "Cheese", "Cottage cheese", "0% fromage blanc", "Goat's cheese", "Feta cheese", "Grated cheese", "Starch", "Corn starch", "Gavotte", "Ginger", "Ginger powder", "Giraumon", "Clove(s)", "Chocolate ice cream", "Gnocchis", "Gorgonzola", "Chia seeds", "Pumpkin seeds", "Cumin seeds", "Sesame seeds", "White sesame seeds", "Golden sesame seeds", "Goose fat", "Coarse salt", "Gruyère", "Grated Gruyère", "Guanciale", "Poultry gizzard(s)", "Green bean(s)", "White beans", "Pink beans", "Kidney beans", "Green bean(s)", "Herbs", "Dried herbs", "Ham", "Cooked ham", "Parma ham", "Jeunes pousses d'épinard", "Ketchup", "Lambis", "Rabbit", "Smoked bacon", "Bacon", "Smoked bacon", "Lenses", "Coral lentils", "Green lentils", "Raw linguine", "Litchi(s)", "Macaronis", "Maïzena", "Mango(s)", "Cassava", "Margarine", "Melted margarine", "Vegetable margarine", "Mascarpone", "Mayonnaise", "Corn", "Maïzena", "McVitties", "Breadcrumbs", "Honey", "Truffle honey", "Mini carottes", "Mirabelles", "Mirin", "Dried morel(s)", "Salt cod", "Mussels", "Mousseron(s)", "Mustard", "Strong mustard", "Old-fashioned mustard", "Mozzarella", "Grated mozzarella", "Nutmeg", "Myrtilles", "4-spice blend", "Nesquik", "Hazelnuts", "Shelled hazelnuts", "Chopped hazelnuts", "Hazelnut powder", "Whole hazelnuts", "Walnuts", "Crushed nuts", "Grated coconut", "Grenoble walnut", "Scallops", "Nutmeg", "Noodles", "Nutella", "Cod eggs", "Yellow onion(s)", "Green olives", "Olives", "Black olives", "Green olives", "Oregano", "Stale bread", "Beef chuck", "Papaya", "Parmesan", "Grated Parmesan", "Sweet potato(s)", "Pecorino", "Pennes", "Chopped parsley", "Chopped parsley", "Chopped parsley", "Petits beurres", "Peas", "Philadelphia", "Pine pinions", "Espelette pepper", "Cayenne pepper", "Sweet pepper powder", "Chilli powder", "Guinea fowl", "Pistachios", "Chopped pistachios", "Unsalted pistachios", "Pizzoccheri", "Oyster mushroom(s)", "Pears in syrup", "Split peas", "Chickpeas", "Fish", "Lamb belly", "Bell pepper(s)", "Bintje potato", "Potatoes", "Pork", "Ground pork", "Pumpkin(s)", "Almond powder", "Cocoa powder", "Yellow curry powder", "Hazelnut powder", "Pistachio powder", "Almond powder", "Pousse d'épinard", "Bamboo shoot", "Soybean pods", "Pralinoise", "Prune(s)", "Dried prunes", "Tomato pulp", "Raspberry puree", "Mash mango", "Mash passion", "Mash tomato", "Hazelnut paste", "Pistachio paste", "Rice paste", "Pasta", "Pépites de chocolat", "Pépites de chocolat noir", "Peaches in syrup", "Quinoa", "Dried grapes", "Ras-el-hanout", "Reblochon", "Leftover meat", "Rhubarb", "Ricotta", "Rigatonis", "Rice", "Arborio rice", "Sticky rice", "Long rice", "Round rice", "Plain puffed rice", "Thaï rice", "Risotto rice", "Roquefort", "Rocket salad", "Brittany shortbreads", "Safran", "Philadelphia", "Sainte-maure de touraine", "Buckwheat", "Savory", "Chocolate sauce", "Soy sauce", "Tomato sauce", "Yakitori sauce", "Smoked sausage(s)", "Sausage", "Salmon", "Cuttlefish", "Saddle of lamb", "Semolina", "Fine semolina", "Shiitake(s)", "Soybeans", "Oat bran", "Soup", "Spaghettis", "Spéculoos", "Sugar", "Brown sugar", "Complete sugar", "Crystal sugar", "Cane sugar", "Powdered sugar", "Powdered sugar", "Brown sugar", "Semolina sugar", "Sesame", "Tagliatelles", "Tahini", "Ham heel", "Tapioca", "Tuna", "Tomate(s) pelée(s)", "Chopped tomatoes", "Dried tomatoes", "Jerusalem artichoke", "Black truffle(s)", "Smoked trout", "Vanilla powder", "Vergeoise blonde", "Chinese vermicelli", "Rice vermicelli", "Meat", "Beef", "Ground beef", "Lean beef", "Minced meat", "Poultry", "Yogurt", "Chopped shallot(s)", "Sweetener", "Pork shoulder", "Chopped spinach", "Spinach(s)", "Dehydrated garlic", "Garlic powder", "Roasted almonds", "Baies roses", "Baies", "Bicarbonate of soda", "Bulot(s)", "Bûches chèvre", "Bitter cocoa powder", "Coffee", "Cinnamon powder", "Caramel", "Salted butter caramel", "Caviar", "Chocolate powder", "Strawberry jam", "Chopped coriander", "Coriander powder", "Corned-beef", "Croutons", "Cumin powder", "Capers", "Spices", "Vanilla essence", "Chopped tarragon", "Vanilla extract", "Poultry fillets", "Edible flowers", "Orange blossom", "Salt flowers", "Red fruit", "Red fruit", "Trim", "Vanilla ice cream", "Fennel seeds", "Mustard seeds", "Harissa", "Provence herbs", "Philadelphia", "Merguez", "Acacia honey", "Morel(s)", "Chocolate mousse", "Nutmeg powder", "Grated nutmeg", "5-berry mix", "Butter hazelnut", "Roasted hazelnuts", "Nutmeg", "Nutmeg powder", "Grated nutmeg", "Nuoc mâm", "Clam(s)", "Paprika", "Smoked paprika", "Perles du japon(s)", "Truffle beads", "Curled parsley", "Sweet peas", "Pork belly", "Pepper", "White pepper", "Lebanese pepper", "Black pepper", "Porto", "Chicken", "Peanut paste", "Green curry paste", "Raviole dough", "Bolognese sauce", "Nuoc mam sauce", "Sausage(s)", "Salt", "Smen", "Blond sugar", "Grainy sugar", "White sesame", "Tabasco", "Cabbage", "Chocolate vermicelli", "Ground meat 5% (fat)", "Wasabi", "Orange zests"]
l = ["Beef broth", "Vegetable broth", "Poultry broth", "Coconut cream", "Liquid fresh cream", "Whole cream", "Semi-thick fresh cream", "Heavy cream", "15% liquid cream", "30% cream", "Light cream", "Orange blossom water", "Poultry stock", "Veal stock", "Poultry stock", "Fish stock", "Oil", "Olive oil", "Coconut oil", "Rapeseed oil", "Walnut oil", "Sesame oil", "Sunflower oil", "Vegetable oil", "Orange juice", "Lemon juice", "Milk", "Unsweetened condensed milk", "Almond milk", "Goat's milk", "Coconut milk", "Soy milk", "Semi-skimmed milk", "Whole milk", "Plant milk", "Skim milk", "Teriyaki sauce", "Agave syrup", "Maple syrup", "Liquid vanilla", "White wine", "Dry white wine", "Red wine", "Amaretto", "Armagnac", "Bitter almond aroma", "Vanilla flavor", "Broth", "Cognac", "Colombo", "Food coloring red", "Whipped cream", "Balsamic vinegar cream with toscoro berries", "35% liquid cream", "Thick cream", "Cuttlefish ink", "Yellow food coloring drops", "Red food coloring drop(s)", "Grand marnier", "Peanut oil", "Frying oil", "Lime juice", "Lychee juice", "Grapefruit juice", "Veal juice", "Rhum", "Dark rum", "Vermouth", "Madeira wine", "Yellow wine", "Rosé wine", "Vinegar", "Balsamic vinegar", "White balsamic vinegar", "White vinegar", "Spirit vinegar", "Cider vinegar", "Rice vinegar", "Red wine vinegar", "Sherry vinegar", "Whisky"]
value = ["Chicken wing(s)", "Victoria pineapple", "Andouillette(s)", "Eggplant(s)", "Avocado(s)", "Banana(s)", "Tray(s) of button mushrooms", "Cherry tomato tray(s)", "Nice apples", "Thick slices of farmhouse bread", "Spoon cookies", "White egg(s)", "White leek(s)", "Jar of pickled bell pepper(s)", "Vegetable broth cube", "Chicken broth", "Celery stalk(s)", "Brick(s) of liquid crème fraîche", "Brocoli(s)", "Fennel bulb", "Bâtons citronnelle", "Bâtons surimis", "Bâtonnets surimis", "Goat's cheese log", "Camembert", "Carottes", "Fresh square", "Carrés chocolat noir", "Chipolatas", "Mild chorizo", "White cabbage", "Kohlrabi", "Lime(s)", "Clove(s)", "Lamb collar(s)", "Combava", "Cucumber(s)", "Pickles", "Zucchini(s)", "Broth cube(s)", "Beef bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Rabbit legs", "Chicken thighs", "Lamb chop", "Artichoke heart", "Back of fresh salmon", "Endives", "Beef rib steak", "Shoulder(s) of lamb", "Turkey cutlet(s)", "Chicken cutlet(s)", "Star anise", "Fennel", "Brick pastry", "Gelatin sheets", "Lasagne sheet(s)", "Bay leaf(s)", "Filo sheets", "Rice leaves", "White figs", "Smoked tenderloin", "Beef tenderloin", "Tournedos filet", "Pike-perch fillets", "Sole fillets", "Pie crust", "Passion fruit(s)", "Patties", "Rice cakes", "Gambas", "Gouttes de tabasco", "Granny", "Grappes de tomates", "Egg yolk(s)", "Khakies", "Kiwi(s)", "Kubor", "Lobsters", "Langoustines", "Manicotti", "Mackerel(s)", "Maroilles", "Mini snickers", "Pieces of sugar", "Turnip(s)", "White onion(s)", "Sweet onion(s)", "Chopped onion(s)", "Oignon(s) moyen(s)", "Red onion(s)", "Green onion(s)", "Chopped onion(s)", "Pitted black olives", "Orange(s)", "Orange(s)", "Oreos", "Hamburger bun(s)", "Grapefruit", "Parsnips", "Boudoir package(s)", "Marshmallow pack(s)", "Wafer pack(s)", "Pack(s) of bacon", "Package(s) smoked bacon", "Powdered hazelnut pack(s)", "Paupiette(s)", "Rump steak(s)", "Salmon steak(s)", "Small peppers", "Small green peppers", "Small red peppers", "Small onion(s)", "Small apples", "Small tomatoes", "Pigeon", "Dried pepper", "Green pepper", "Red pepper(s)", "Dragon fruit", "Praline plate(s)", "Pear(s)", "Leek(s)", "Yellow bell pepper(s)", "Red bell pepper(s)", "Green bell pepper(s)", "Apple(s)", "Potato(es)", "Pumpkin", "Octopus(s)", "Pitted prune(s)", "Shortbread dough", "Pizza dough", "Pie dough", "Pâte(s) brisée(s)", "Pêches", "Quenelle(s)", "Monkfish tail(s)", "Raifort", "Goat's cheese log", "Rouleau pâte(s) feuilletée(s)", "Rumsteak(s)", "Scallops", "Pike-perch", "Sardine", "Sausage(s)", "Tuna steak(s)", "Chopped steak(s)", "Guinea fowl supreme", "Capon supremes", "Garlic and parsley tablet(s)", "Milk chocolate tablet(s)", "White chocolate tablet(s)", "Black chocolate tablet(s)", "Rhubarb stems", "Tomato(s)", "Tomate(s) cerise(s)", "Tomate(s) confite(s)", "Dried tomato(s)", "Medium tomatoes", "Corn tortilla", "Tortillas", "Ham slice(s)", "Slice(s) of Bayonne ham", "Parma ham slice(s)", "Slices of smoked salmon", "Trout(s)", "Twix", "Lemon zests", "Lemon zests", "Shallot(s)", "Minced shallot(s)", "Lamb shoulder", "Egg(s)", "Parsley bouquet", "Garnished bouquet(s)", "Bottle(s) brut champagne", "Bottle(s) red wine", "Can(s) tomato paste", "Lemon(s)", "Crêpes", "Toothpicks", "Onion(s)", "Milk bread", "Chili pepper", "Sweet pepper", "Hot pepper(s)", "Pâte feuilletée", "Garlic sausage"]
bunch = ["Green asparagus(s)", "Rosemary"]
slice = ["Bacon", "Baguette", "Cheddar", "Swordfish", "Leg of lamb", "Bread", "Brioche bread", "Gingerbread", "Country bread", "Crumb bread"]
bunch1 = ["Chervil", "Menthe", "Parsley", "Flat-leaf parsley"]
leaf = ["Basil", "Gelatin", "Lettuce", "Lasagnes", "Laurel", "Fresh mint", "Filo dough", "Salad", "Sauge", "Wraps"]
sprig = ["Dill", "Thyme"]
box = ["Flageolet(s)", "Garlic and herb cheese", "Condensed milk", "Sweetened condensed milk", "Litchi in syrup", "Mutti garlic polpa"]
sprig1 = ["Chives", "Coriander", "Tarragon"]
bag = ["Cream pudding", "Crêpes dentelles", "Instant jelly", "Yeast", "Baker's yeast", "Baking powder", "Frozen vegetable medley", "Vanilla sugar", "Tortellinis"]
pod = ["Garlic", "Cardamom", "Vanilla"]

units = kg
units += l + value + bunch + slice + bunch1 + leaf + sprig + box + sprig1 + bag + pod
print(len(units))

792


In [36]:
# DON'T NEED TO BE COMPUTED

item_units = {
    "kg": ["Dried apricots", "Agar-agar", "Amandes concassées", "Amandes en poudre vahiné", "Amandes", "Slivered almonds", "Vanilla slivered almonds", "Powdered almonds", "Pineapple", "Chopped dill", "Lemon vanilla flavor", "White asparagus", "Beet(s)", "Red beet(s)", "Butter", "Peanut butter", "Semi-salted butter", "Soft butter", "Salted butter", "Soft butter", "Biscuits", "Bisques de homard", "Chicken breast(s)", "Chard(s)", "Boursin(s)", "Blue Bresse", "Brioche", "Burrata", "Beef", "Ground beef", "Cacahuètes", "Bitter cocoa", "Cocoa powder", "Unsweetened cocoa powder", "Soluble coffee", "Duck(s)", "Cancoillotte", "Cinnamon", "Brown sugar", "Cherries", "Walnut kernels", "Sausage meat", "Champignon(s)", "Champignon(s) de paris", "Mushrooms", "Paris mushrooms", "Chanterelle(s)", "Chantilly", "Breadcrumbs", "Chocolate", "Bitter chocolate powder", "Milk chocolate", "White chocolate", "Dark chocolate", "50% dark chocolate", "Dark pastry chocolate", "70% cocoa dark chocolate", "Dark baking chocolate", "Pastry chocolate", "Chorizo", "Strong chorizo", "Brussels sprouts", "Green cabbage", "Cabbage(s)", "Chou(x) de bruxelle", "Cauliflower", "Chestnut(s)", "Goat cheese", "Heart of palm", "Quince", "Lamb collar", "Apple compote", "Comté cheese", "Shredded Comté cheese", "Tomato concentrate", "Conchiglionis", "Raspberry jam", "Chocolate chips", "Parmesan shavings", "Hulls", "Corn flakes", "Raspberry sauce", "Red fruit coulis", "Tomato coulis", "Butternut squash", "Couscous", "Crab(s)", "Crackers salés", "Dried cranberries", "Cranberrys", "Shrimp(s)", "Shrimp(s)", "Grey shrimps", "Pink shrimps", "Crème", "Light cream", "Custard cream", "Chestnut cream", "Whole cream", "30% full cream", "Whipping cream", "Crème fleurette 30%", "Whipped cream", "Heavy cream", "Fresh cream", "30% fresh cream", "30% fresh cream", "3% light cream", "Whole cream", "Mounted cream", "Sour cream", "Crêpe dentelle(s)", "Cumin", "Turmeric", "Curry", "Capers", "Porcini mushrooms", "Celery root", "Medjool date", "Dates", "Medjool date", "Dried dates", "Diced ham", "Emmental", "Grated Emmental cheese", "Squid(s)", "Fiscella", "Flour", "Buckwheat flour", "Wheat flour", "Rice flour", "Sticky rice flour", "Sticky rice flour", "Flour t55", "Sifted flour", "Feta cheese", "Figs", "Dried figs", "Beef tenderloin", "Pork tenderloin", "Pork tenderloin", "Salmon fillet(s)", "Herbs", "Oat flakes", "Foie gras", "Strawberry(s)", "Tagada strawberry(s)", "Raspberry(s)", "Cheese", "Cottage cheese", "0% fromage blanc", "Goat's cheese", "Feta cheese", "Grated cheese", "Starch", "Corn starch", "Gavotte", "Ginger", "Ginger powder", "Giraumon", "Clove(s)", "Chocolate ice cream", "Gnocchis", "Gorgonzola", "Chia seeds", "Pumpkin seeds", "Cumin seeds", "Sesame seeds", "White sesame seeds", "Golden sesame seeds", "Goose fat", "Coarse salt", "Gruyère", "Grated Gruyère", "Guanciale", "Poultry gizzard(s)", "Green bean(s)", "White beans", "Pink beans", "Kidney beans", "Green bean(s)", "Herbs", "Dried herbs", "Ham", "Cooked ham", "Parma ham", "Jeunes pousses d'épinard", "Ketchup", "Lambis", "Rabbit", "Smoked bacon", "Bacon", "Smoked bacon", "Lenses", "Coral lentils", "Green lentils", "Raw linguine", "Litchi(s)", "Macaronis", "Maïzena", "Mango(s)", "Cassava", "Margarine", "Melted margarine", "Vegetable margarine", "Mascarpone", "Mayonnaise", "Corn", "Maïzena", "McVitties", "Breadcrumbs", "Honey", "Truffle honey", "Mini carottes", "Mirabelles", "Mirin", "Dried morel(s)", "Salt cod", "Mussels", "Mousseron(s)", "Mustard", "Strong mustard", "Old-fashioned mustard", "Mozzarella", "Grated mozzarella", "Nutmeg", "Myrtilles", "4-spice blend", "Nesquik", "Hazelnuts", "Shelled hazelnuts", "Chopped hazelnuts", "Hazelnut powder", "Whole hazelnuts", "Walnuts", "Crushed nuts", "Grated coconut", "Grenoble walnut", "Scallops", "Nutmeg", "Noodles", "Nutella", "Cod eggs", "Yellow onion(s)", "Green olives", "Olives", "Black olives", "Green olives", "Oregano", "Stale bread", "Beef chuck", "Papaya", "Parmesan", "Grated Parmesan", "Sweet potato(s)", "Pecorino", "Pennes", "Chopped parsley", "Chopped parsley", "Chopped parsley", "Petits beurres", "Peas", "Philadelphia", "Pine pinions", "Espelette pepper", "Cayenne pepper", "Sweet pepper powder", "Chilli powder", "Guinea fowl", "Pistachios", "Chopped pistachios", "Pistaches non salée(s)", "Pizzoccheri", "Oyster mushroom(s)", "Pears in syrup", "Split peas", "Chickpeas", "Fish", "Lamb belly", "Bell pepper(s)", "Bintje potato", "Potatoes", "Pork", "Ground pork", "Pumpkin(s)", "Almond powder", "Cocoa powder", "Yellow curry powder", "Hazelnut powder", "Pistachio powder", "Almond powder", "Pousses d'épinard", "Bamboo shoot", "Soybean pods", "Pralinoise", "Prune(s)", "Dried prunes", "Tomato pulp", "Raspberry puree", "Mash mango", "Mash passion", "Mash tomato", "Hazelnut paste", "Pistachio paste", "Rice paste", "Pasta", "Pépites de chocolat", "Pépites de chocolat noir", "Peaches in syrup", "Quinoa", "Dried grapes", "Ras-el-hanout", "Reblochon", "Leftover meat", "Rhubarb", "Ricotta", "Rigatonis", "Rice", "Arborio rice", "Sticky rice", "Long rice", "Round rice", "Plain puffed rice", "Thaï rice", "Risotto rice", "Roquefort", "Rocket salad", "Brittany shortbreads", "Safran", "Philadelphia", "Sainte-maure de touraine", "Buckwheat", "Savory", "Chocolate sauce", "Soy sauce", "Tomato sauce", "Yakitori sauce", "Smoked sausage(s)", "Sausage", "Salmon", "Cuttlefish", "Saddle of lamb", "Semolina", "Fine semolina", "Shiitake(s)", "Soybeans", "Oat bran", "Soup", "Spaghettis", "Spéculoos", "Sugar", "Brown sugar", "Complete sugar", "Crystal sugar", "Cane sugar", "Powdered sugar", "Powdered sugar", "Brown sugar", "Semolina sugar", "Sesame", "Tagliatelles", "Tahini", "Ham heel", "Tapioca", "Tuna", "Tomate(s) pelée(s)", "Chopped tomatoes", "Dried tomatoes", "Jerusalem artichoke", "Black truffle(s)", "Smoked trout", "Vanilla powder", "Vergeoise blonde", "Chinese vermicelli", "Rice vermicelli", "Meat", "Beef", "Ground beef", "Lean beef", "Minced meat", "Poultry", "Yogurt", "Chopped shallot(s)", "Sweetener", "Pork shoulder", "Chopped spinach", "Spinach(s)", "Dehydrated garlic", "Garlic powder", "Roasted almonds", "Baies roses", "Baies", "Bicarbonate of soda", "Bulot(s)", "Bûches chèvre", "Bitter cocoa powder", "Coffee", "Cinnamon powder", "Caramel", "Salted butter caramel", "Caviar", "Chocolate powder", "Strawberry jam", "Chopped coriander", "Coriander powder", "Corned-beef", "Croutons", "Cumin powder", "Capers", "Spices", "Vanilla essence", "Chopped tarragon", "Vanilla extract", "Poultry fillets", "Edible flowers", "Orange blossom", "Salt flowers", "Red fruit", "Red fruit", "Trim", "Vanilla ice cream", "Fennel seeds", "Mustard seeds", "Harissa", "Provence herbs", "Philadelphia", "Merguez", "Acacia honey", "Morel(s)", "Chocolate mousse", "Nutmeg powder", "Grated nutmeg", "5-berry mix", "Butter hazelnut", "Roasted hazelnuts", "Nutmeg", "Nutmeg powder", "Grated nutmeg", "Nuoc mâm", "Clam(s)", "Paprika", "Smoked paprika", "Japanese pearls", "Truffle beads", "Curled parsley", "Sweet peas", "Pork belly", "Pepper", "White pepper", "Lebanese pepper", "Black pepper", "Porto", "Chicken", "Peanut paste", "Green curry paste", "Raviole dough", "Bolognese sauce", "Nuoc mam sauce", "Sausage(s)", "Salt", "Smen", "Blond sugar", "Grainy sugar", "White sesame", "Tabasco", "Cabbage", "Chocolate vermicelli", "Ground meat 5% (fat)", "Wasabi", "Orange zests"],
    "l": ["Beef broth", "Vegetable broth", "Poultry broth", "Coconut cream", "Liquid fresh cream", "Whole cream", "Semi-thick fresh cream", "Heavy cream", "15% liquid cream", "30% cream", "Light cream", "Orange blossom water", "Poultry stock", "Veal stock", "Poultry stock", "Fish stock", "Oil", "Olive oil", "Coconut oil", "Rapeseed oil", "Walnut oil", "Sesame oil", "Sunflower oil", "Vegetable oil", "Orange juice", "Lemon juice", "Milk", "Unsweetened condensed milk", "Almond milk", "Goat's milk", "Coconut milk", "Soy milk", "Semi-skimmed milk", "Whole milk", "Plant milk", "Skim milk", "Teriyaki sauce", "Agave syrup", "Maple syrup", "Liquid vanilla", "White wine", "Dry white wine", "Red wine", "Amaretto", "Armagnac", "Bitter almond aroma", "Vanilla flavor", "Broth", "Cognac", "Colombo", "Food coloring red", "Whipped cream", "Balsamic vinegar cream with toscoro berries", "35% liquid cream", "Thick cream", "Cuttlefish ink", "Yellow food coloring drops", "Red food coloring drop(s)", "Grand marnier", "Peanut oil", "Frying oil", "Lime juice", "Lychee juice", "Grapefruit juice", "Veal juice", "Rhum", "Dark rum", "Vermouth", "Madeira wine", "Yellow wine", "Rosé wine", "Vinegar", "Balsamic vinegar", "White balsamic vinegar", "White vinegar", "Spirit vinegar", "Cider vinegar", "Rice vinegar", "Red wine vinegar", "Sherry vinegar", "Whisky"],
    "value": ["Chicken wing(s)", "Victoria pineapple", "Andouillette(s)", "Eggplant(s)", "Avocado(s)", "Banana(s)", "Tray(s) of button mushrooms", "Cherry tomato tray(s)", "Nice apples", "Thick slices of farmhouse bread", "Spoon cookies", "White egg(s)", "White leek(s)", "Jar of pickled bell pepper(s)", "Vegetable broth cube", "Chicken broth", "Celery stalk(s)", "Brick(s) of liquid crème fraîche", "Brocoli(s)", "Fennel bulb", "Bâtons citronnelle", "Bâtons surimis", "Bâtonnets surimis", "Goat's cheese log", "Camembert", "Carottes", "Fresh square", "Carrés chocolat noir", "Chipolatas", "Mild chorizo", "White cabbage", "Kohlrabi", "Lime(s)", "Clove(s)", "Lamb collar(s)", "Combava", "Cucumber(s)", "Pickles", "Zucchini(s)", "Broth cube(s)", "Beef bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Rabbit legs", "Chicken thighs", "Lamb chop", "Artichoke heart", "Back of fresh salmon", "Endives", "Beef rib steak", "Shoulder(s) of lamb", "Turkey cutlet(s)", "Chicken cutlet(s)", "Star anise", "Fennel", "Brick pastry", "Gelatin sheets", "Lasagne sheet(s)", "Bay leaf(s)", "Filo sheets", "Rice leaves", "White figs", "Smoked tenderloin", "Beef tenderloin", "Tournedos filet", "Pike-perch fillets", "Sole fillets", "Pie crust", "Passion fruit(s)", "Patties", "Rice cakes", "Gambas", "Gouttes de tabasco", "Granny", "Grappes de tomates", "Egg yolk(s)", "Khakies", "Kiwi(s)", "Kubor", "Lobsters", "Langoustines", "Manicotti", "Mackerel(s)", "Maroilles", "Mini snickers", "Pieces of sugar", "Turnip(s)", "White onion(s)", "Sweet onion(s)", "Chopped onion(s)", "Oignon(s) moyen(s)", "Red onion(s)", "Green onion(s)", "Chopped onion(s)", "Pitted black olives", "Orange(s)", "Orange(s)", "Oreos", "Hamburger bun(s)", "Grapefruit", "Parsnips", "Boudoir package(s)", "Marshmallow pack(s)", "Wafer pack(s)", "Pack(s) of bacon", "Package(s) smoked bacon", "Powdered hazelnut pack(s)", "Paupiette(s)", "Rump steak(s)", "Salmon steak(s)", "Small peppers", "Small green peppers", "Small red peppers", "Small onion(s)", "Small apples", "Small tomatoes", "Pigeon", "Dried pepper", "Green pepper", "Red pepper(s)", "Dragon fruit", "Praline plate(s)", "Pear(s)", "Leek(s)", "Yellow bell pepper(s)", "Red bell pepper(s)", "Green bell pepper(s)", "Apple(s)", "Potato(es)", "Pumpkin", "Octopus(s)", "Pitted prune(s)", "Shortbread dough", "Pizza dough", "Pie dough", "Pâte(s) brisée(s)", "Pêches", "Quenelle(s)", "Monkfish tail(s)", "Raifort", "Goat's cheese log", "Rouleau pâte(s) feuilletée(s)", "Rumsteak(s)", "Scallops", "Pike-perch", "Sardine", "Sausage(s)", "Tuna steak(s)", "Chopped steak(s)", "Guinea fowl supreme", "Capon supremes", "Garlic and parsley tablet(s)", "Milk chocolate tablet(s)", "White chocolate tablet(s)", "Black chocolate tablet(s)", "Rhubarb stems", "Tomato(s)", "Tomate(s) cerise(s)", "Tomate(s) confite(s)", "Dried tomato(s)", "Medium tomatoes", "Corn tortilla", "Tortillas", "Ham slice(s)", "Slice(s) of Bayonne ham", "Parma ham slice(s)", "Slices of smoked salmon", "Trout(s)", "Twix", "Lemon zests", "Lemon zests", "Shallot(s)", "Minced shallot(s)", "Lamb shoulder", "Egg(s)", "Parsley bouquet", "Garnished bouquet(s)", "Bottle(s) brut champagne", "Bottle(s) red wine", "Can(s) tomato paste", "Lemon(s)", "Crêpes", "Toothpicks", "Onion(s)", "Milk bread", "Chili pepper", "Sweet pepper", "Hot pepper(s)", "Pâte feuilletée", "Garlic sausage"],
    "bunch": ["Green asparagus(s)", "Rosemary"],
    "slice": ["Bacon", "Baguette", "Cheddar", "Swordfish", "Leg of lamb", "Bread", "Brioche bread", "Gingerbread", "Country bread", "Crumb bread"],
    "bunch1": ["Chervil", "Menthe", "Parsley", "Flat-leaf parsley"],
    "leaf": ["Basil", "Gelatin", "Lettuce", "Lasagnes", "Laurel", "Fresh mint", "Filo dough", "Salad", "Sauge", "Wraps"],
    "sprig": ["Dill", "Thyme"],
    "box": ["Flageolet(s)", "Garlic and herb cheese", "Condensed milk", "Sweetened condensed milk", "Litchi in syrup", "Mutti garlic polpa"],
    "sprig1": ["Chives", "Coriander", "Tarragon"],
    "sachet": ["Cream pudding", "Crêpes dentelles", "Instant jelly", "Yeast", "Baker's yeast", "Baking powder", "Frozen vegetable medley", "Vanilla sugar", "Tortellinis"],
    "pod": ["Garlic", "Cardamom", "Vanilla"]
}

def trouver_tableau_contenant_string(dictionnaire_tableaux, string_a_chercher):
    for nom_tableau, tableau in dictionnaire_tableaux.items():
        if string_a_chercher in tableau:
            return nom_tableau
    return "Not find"

item_units["kg"]

['Dried apricots',
 'Agar-agar',
 'Amandes concassées',
 'Amandes en poudre vahiné',
 'Amandes',
 'Slivered almonds',
 'Vanilla slivered almonds',
 'Powdered almonds',
 'Pineapple',
 'Chopped dill',
 'Lemon vanilla flavor',
 'White asparagus',
 'Beet(s)',
 'Red beet(s)',
 'Butter',
 'Peanut butter',
 'Semi-salted butter',
 'Soft butter',
 'Salted butter',
 'Soft butter',
 'Biscuits',
 'Bisques de homard',
 'Chicken breast(s)',
 'Chard(s)',
 'Boursin(s)',
 'Blue Bresse',
 'Brioche',
 'Burrata',
 'Beef',
 'Ground beef',
 'Cacahuètes',
 'Bitter cocoa',
 'Cocoa powder',
 'Unsweetened cocoa powder',
 'Soluble coffee',
 'Duck(s)',
 'Cancoillotte',
 'Cinnamon',
 'Brown sugar',
 'Cherries',
 'Walnut kernels',
 'Sausage meat',
 'Champignon(s)',
 'Champignon(s) de paris',
 'Mushrooms',
 'Paris mushrooms',
 'Chanterelle(s)',
 'Chantilly',
 'Breadcrumbs',
 'Chocolate',
 'Bitter chocolate powder',
 'Milk chocolate',
 'White chocolate',
 'Dark chocolate',
 '50% dark chocolate',
 'Dark pastry chocola

In [ ]:
# DON'T NEED TO BE COMPUTED

def find_table_containing_string(dictionnaire_tableaux, string_a_chercher):
    for nom_tableau, tableau in dictionnaire_tableaux.items():
        if string_a_chercher in tableau:
            return nom_tableau
    return "Not find:" + string_a_chercher

item_categories = [
    
    ("Meats:", ["Chicken wing(s)", "Andouillette(s)", "Bacon", "Chicken breast(s)", "Beef", "Ground beef", "Duck(s)", "Sausage meat", "Chipolatas", "Chorizo", "Mild chorizo", "Strong chorizo", "Lamb collars", "Corned-beef", "Rabbit legs", "Chicken thighs", "Lamb chop", "Diced ham", "Beef rib steak", "Shoulder(s) of lamb", "Turkey cutlets", "Chicken cutlets", "Beef tenderloin", "Pork tenderloin", "Pork tenderloin", "Smoked tenderloin", "Beef tenderloin", "Poultry fillets", "Foie gras", "Poultry stock", "Veal stock", "Poultry stock", "Leg of lamb", "Goose fat", "Guanciale", "Poultry gizzard(s)", "Ham", "Cooked ham", "Parma ham", "Veal juice", "Rabbit", "Smoked bacon", "Bacon", "Smoked bacon", "Merguez", "Beef chuck", "Clam(s)", "Pack(s) of bacon", "Package(s) smoked bacon", "Paupiettes", "Rump steak(s)", "Pigeon", "Guinea fowl", "Lamb belly", "Pork belly", "Pork", "Ground pork", "Chicken", "Leftover meat", "Rumsteak(s)", "Sausage(s)", "Sausage(s)", "Smoked sausage(s)", "Sausage", "Garlic sausage", "Saddle of lamb", "Chopped steak(s)", "Guinea fowl supreme", "Capon supremes", "Ham heel", "Slices of ham", "Slice(s) of Bayonne ham", "Parma ham slice(s)", "Meat", "Beef", "Ground beef", "Lean beef", "Minced meat", "Ground meat 5% (fat)", "Poultry", "Lamb shoulder", "Pork shoulder"]),
    
    ("Drinks:", ["Amaretto", "Armagnac", "Bottle(s) brut champagne", "Bottle(s) red wine", "Cognac", "Colombo", "Grand marnier", "Orange juice", "Lemon juice", "Lime juice", "Lychee juice", "Grapefruit juice", "Porto", "Rhum", "Dark rum", "Vergeoise blonde", "Vermouth", "White wine", "Dry white wine", "Madeira wine", "Yellow wine", "Rosé wine", "Red wine", "Whisky"]),
    ("Vegetables:", ["Garlic", "Dehydrated garlic", "Garlic powder", "White asparagus", "Green asparagus(s)", "Eggplant(s)", "Avocado(s)", "Tray(s) of button mushrooms", "Cherry tomato tray(s)", "Beet(s)", "Red beet(s)", "White leek(s)", "Chard(s)", "Jar of pickled bell pepper(s)", "Can(s) tomato paste", "Brocoli(s)", "Carottes", "Mushrooms", "Paris mushrooms", "Chanterelle(s)", "White cabbage", "Brussels sprouts", "Kohlrabi", "Green cabbage", "Cabbage(s)", "Chou(x) de bruxelle", "Cauliflower", "Heart of palm", "Combava", "Tomato concentrate", "Cucumber(s)", "Pickles", "Tomato coulis", "Butternut squash", "Zucchini(s)", "Porcini mushrooms", "Artichoke heart", "Endives", "Flageolet(s)", "Giraumon", "Clove(s)", "Grappes de tomates", "White beans", "Pink beans", "Kidney beans", "Green bean(s)", "Jeunes pousses d'épinard", "Lettuce", "Lenses", "Coral lentils", "Green lentils", "Frozen vegetable medley", "Cassava", "Corn", "Mini carottes", "Mousseron(s)", "Turnip(s)", "Onion(s)", "White onion(s)", "Sweet onion(s)", "Chopped onion(s)", "Yellow onion(s)", "Oignon(s) moyen(s)", "Red onion(s)", "Green onion(s)", "Chopped onion(s)", "Green olives", "Olives", "Black olives", "Pitted black olives", "Green olives", "Parsnips", "Sweet potato(s)", "Truffle beads", "Small peppers", "Small green peppers", "Small red peppers", "Small onion(s)", "Small tomatoes", "Peas", "Chili pepper", "Espelette pepper", "Cayenne pepper", "Sweet pepper", "Sweet pepper powder", "Chilli powder", "Hot pepper(s)", "Dried pepper", "Green pepper", "Red pepper(s)", "Oyster mushroom(s)", "Leek(s)", "Pears in syrup", "Split peas", "Chickpeas", "Sweet peas", "Bell pepper(s)", "Yellow bell pepper(s)", "Red bell pepper(s)", "Green bell pepper(s)", "Pumpkin", "Pumpkin(s)", "Pousses d'épinard", "Bamboo shoot", "Soybean pods", "Tomato pulp", "Mash tomato", "Ras-el-hanout", "Rocket salad", "Salad", "Shiitake(s)", "Soybeans", "Tapioca", "Tomato(s)", "Tomate(s) cerise(s)", "Tomate(s) confite(s)", "Tomate(s) pelée(s)", "Dried tomato(s)", "Chopped tomatoes", "Medium tomatoes", "Dried tomatoes", "Jerusalem artichoke", "Black truffle(s)", "Cabbage", "Shallot(s)", "Chopped shallot(s)", "Minced shallot(s)", "Chopped spinach", "Spinach(s)"]),
    ("Herbs and spices:", ["Dill", "Chopped dill", "Basil", "Parsley bouquet", "Garnished bouquet(s)", "Celery stalk(s)", "Fennel bulb", "Bâtons citronnelle", "Cinnamon", "Cinnamon powder", "Chervil", "Chives", "Clove(s)", "Coriander", "Chopped coriander", "Coriander powder", "Cumin", "Cumin powder", "Turmeric", "Curry", "Celery root", "Spices", "Chopped tarragon", "Fennel", "Bay leaf(s)", "Filo sheets", "Rice leaves", "Herbs", "Edible flowers", "Salt flowers", "Trim", "Ginger", "Ginger powder", "Fennel seeds", "Mustard seeds", "Coarse salt", "Herbs", "Provence herbs", "Dried herbs", "Kubor", "Laurel", "Menthe", "Fresh mint", "4-spice blend", "5-berry mix", "Oregano", "Paprika", "Smoked paprika", "Parsley", "Chopped parsley", "Curled parsley", "Chopped parsley", "Chopped parsley", "Flat-leaf parsley", "Pepper", "White pepper", "Lebanese pepper", "Black pepper", "Yellow curry powder", "Rosemary", "Safran", "Buckwheat", "Savory", "Sauge", "Salt", "Garlic and parsley tablet(s)", "Thyme", "Rhubarb stems", "Wasabi"]),
    ("Fishes:", ["Bisques de homard", "Bulot(s)", "Bâtons surimis", "Bâtonnets surimis", "Caviar", "Crab(s)", "Shrimp(s)", "Grey shrimps", "Pink shrimps", "Back of fresh salmon", "Cuttlefish ink", "Swordfish", "Tarragon", "Tournedos filet", "Salmon fillet(s)", "Pike-perch fillets", "Sole fillets", "Fish stock", "Gambas", "Lambis", "Lobsters", "Langoustines", "Mackerel(s)", "Morel(s)", "Dried morel(s)", "Salt cod", "Mussels", "Scallops", "Nuoc mâm", "Cod eggs", "Salmon steak(s)", "Fish", "Mutti garlic polpa", "Octopus(s)", "Quenelle(s)", "Monkfish tail(s)", "Scallops", "Pike-perch", "Sardine", "Salmon", "Cuttlefish", "Soup", "Tuna steak(s)", "Tuna", "Slices of smoked salmon", "Smoked trout", "Trout(s)"]),
    ("Dough:", ["Peanut paste", "Green curry paste", "Hazelnut paste", "Pistachio paste", "Rice paste", "Pâte feuilletée", "Filo dough", "Shortbread dough", "Pizza dough", "Raviole dough", "Pie dough", "Pâte(s) brisée(s)", "Pasta"]),
    ("Milk item:", ["Boursin(s)", "Blue Bresse", "Brick(s) of liquid crème fraîche", "Burrata", "Goat's cheese log", "Bûches chèvre", "Camembert", "Cancoillotte", "Fresh square", "Chantilly", "Cheddar", "Goat cheese", "Comté cheese", "Shredded Comté cheese", "Parmesan shavings", "Crème", "Light cream", "Custard cream", "Whipped cream", "Coconut cream", "Chestnut cream", "Balsamic vinegar cream with toscoro berries", "Whole cream", "30% full cream", "Whipping cream", "Crème fleurette 30%", "Whipped cream", "Heavy cream", "Fresh cream", "30% fresh cream", "30% fresh cream", "3% light cream", "Whole cream", "Liquid fresh cream", "Whole cream", "Semi-thick fresh cream", "Heavy cream", "15% liquid cream", "35% liquid cream", "30% cream", "Light cream", "Mounted cream", "Cream pudding", "Sour cream", "Thick cream", "Emmental", "Grated Emmental cheese", "Fiscella", "Feta cheese", "Cheese", "Garlic and herb cheese", "Cottage cheese", "0% fromage blanc", "Goat's cheese", "Feta cheese", "Grated cheese", "Chocolate ice cream", "Vanilla ice cream", "Gorgonzola", "Gruyère", "Grated Gruyère", "Philadelphia", "Milk", "Condensed milk", "Unsweetened condensed milk", "Sweetened condensed milk", "Almond milk", "Goat's milk", "Coconut milk", "Soy milk", "Semi-skimmed milk", "Whole milk", "Plant milk", "Skim milk", "Manicotti", "Maroilles", "Mascarpone", "Mozzarella", "Grated mozzarella", "Parmesan", "Grated Parmesan", "Pecorino", "Philadelphia", "Pizzoccheri", "Raifort", "Reblochon", "Ricotta", "Goat's cheese log", "Roquefort", "Philadelphia", "Sainte-maure de touraine", "Yogurt"]),
    ("Bakery:", ["Lemon vanilla flavor", "Bitter almond aroma", "Vanilla flavor", "Butter", "Peanut butter", "Semi-salted butter", "Soft butter", "Salted butter", "Soft butter", "Bicarbonate of soda", "Biscuits", "Spoon cookies", "Bitter cocoa", "Bitter cocoa powder", "Cocoa powder", "Unsweetened cocoa powder", "Caramel", "Salted butter caramel", "Carrés chocolat noir", "Brown sugar", "Chocolate", "Bitter chocolate powder", "Milk chocolate", "White chocolate", "Chocolate powder", "Dark chocolate", "50% dark chocolate", "Dark pastry chocolate", "70% cocoa dark chocolate", "Dark baking chocolate", "Pastry chocolate", "Food coloring red", "Chocolate chips", "Orange blossom water", "Vanilla essence", "Vanilla extract", "Flour", "Buckwheat flour", "Wheat flour", "Rice flour", "Sticky rice flour", "Sticky rice flour", "Flour t55", "Sifted flour", "Gelatin sheets", "Orange blossom", "Yellow food coloring drops", "Red food coloring drop(s)", "Gelatin", "Yeast", "Baker's yeast", "Baking powder", "Margarine", "Melted margarine", "Vegetable margarine", "Honey", "Acacia honey", "Truffle honey", "Mini snickers", "Pieces of sugar", "Nesquik", "Butter hazelnut", "Nutella", "Oreos", "Boudoir package(s)", "Wafer pack(s)", "Petits beurres", "Praline plate(s)", "Almond powder", "Cocoa powder", "Pralinoise", "Pépites de chocolat", "Pépites de chocolat noir", "Brittany shortbreads", "Agave syrup", "Maple syrup", "Smen", "Spéculoos", "Sugar", "Blond sugar", "Brown sugar", "Complete sugar", "Crystal sugar", "Cane sugar", "Powdered sugar", "Powdered sugar", "Grainy sugar", "Brown sugar", "Semolina sugar", "Vanilla sugar", "Milk chocolate tablet(s)", "White chocolate tablet(s)", "Black chocolate tablet(s)", "Twix", "Vanilla", "Vanilla powder", "Liquid vanilla", "Chocolate vermicelli", "Sweetener"]),
    ("Others:", ["White egg(s)", "Broth", "Vegetable broth cube", "Beef broth", "Vegetable broth", "Chicken broth", "Poultry broth", "Cacahuètes", "Coffee", "Soluble coffee", "Breadcrumbs", "Hulls", "Corn flakes", "Crackers salés", "Croutons", "Crêpes dentelles", "Crêpes", "Sachet(s) de crêpes dentelles", "Broth cube(s)", "Beef bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Toothpicks", "Squid(s)", "Brick pastry", "Lasagne sheet(s)", "Oat flakes", "Pie crust", "Tagada strawberry(s)", "Starch", "Corn starch", "Patties", "Rice cakes", "Gavotte", "Instant jelly", "Granny", "Oil", "Peanut oil", "Olive oil", "Coconut oil", "Rapeseed oil", "Frying oil", "Walnut oil", "Sesame oil", "Sunflower oil", "Vegetable oil", "Egg yolk(s)", "Macaronis", "Maïzena", "Maïzena", "McVitties", "Breadcrumbs", "Mirin", "Chocolate mousse", "Bread", "Milk bread", "Brioche bread", "Gingerbread", "Country bread", "Crumb bread", "Stale bread", "Hamburger bun(s)", "Marshmallow pack(s)", "Japanese pearls", "Oat bran", "Corn tortilla", "Tortillas", "Vinegar", "Balsamic vinegar", "White balsamic vinegar", "White vinegar", "Spirit vinegar", "Cider vinegar", "Rice vinegar", "Red wine vinegar", "Sherry vinegar", "Wraps", "Egg(s)"]),
    ("Sides:", ["Baguette", "Thick slices of farmhouse bread", "Brioche", "Conchiglionis", "Couscous", "Gnocchis", "Lasagnes", "Raw linguine", "Noodles", "Pennes", "Bintje potato", "Potato(es)", "Potatoes", "Quinoa", "Rigatonis", "Rice", "Arborio rice", "Sticky rice", "Long rice", "Round rice", "Plain puffed rice", "Thaï rice", "Risotto rice", "Semolina", "Fine semolina", "Spaghettis", "Tagliatelles", "Tortellinis", "Chinese vermicelli", "Rice vermicelli"]),
    ("Sauces:", ["Harissa", "Ketchup", "Mayonnaise", "Mustard", "Strong mustard", "Old-fashioned mustard", "Bolognese sauce", "Chocolate sauce", "Nuoc mam sauce", "Soy sauce", "Teriyaki sauce", "Tomato sauce", "Yakitori sauce", "Tabasco", "Tahini", "Tabasco drops"]),
    ("Fruits:", ["Dried apricots", "Agar-agar", "Pineapple", "Victoria pineapple", "Baies roses", "Baies", "Banana(s)", "Nice apples", "Cardamom", "Cherries", "Lemon(s)", "Lime(s)", "Quince", "Apple compote", "Strawberry jam", "Raspberry jam", "Raspberry sauce", "Red fruit coulis", "Dried cranberries", "Cranberrys", "Capers", "Dates", "Medjool date", "Dried dates", "Star anise", "Figs", "White figs", "Dried figs", "Strawberry(s)", "Raspberry(s)", "Passion fruit(s)", "Red fruit", "Red fruit", "Khakies", "Kiwi(s)", "Litchi in syrup", "Litchi(s)", "Mango(s)", "Mirabelles", "Myrtilles", "Grated coconut", "Orange(s)", "Grapefruit", "Papaya", "Small apples", "Dragon fruit", "Pear(s)", "Apple(s)", "Prune(s)", "Pitted prune(s)", "Dried prunes", "Raspberry puree", "Mash mango", "Mash passion", "Peaches", "Peaches in syrup", "Dried grapes", "Rhubarb", "Lemon zests", "Orange zests"]),
    ("Nuts:", ["Chopped almonds", "Vanilla almond powder", "Almonds", "Slivered almonds", "Vanilla slivered almonds", "Powdered almonds", "Roasted almonds", "Walnut kernels", "Chia seeds", "Pumpkin seeds", "Cumin seeds", "Sesame seeds", "White sesame seeds", "Golden sesame seeds", "Nutmeg", "Nutmeg powder", "Grated nutmeg", "Hazelnuts", "Shelled hazelnuts", "Roasted hazelnuts", "Chopped hazelnuts", "Hazelnut powder", "Whole hazelnuts", "Walnuts", "Crushed nuts", "Crushed nuts", "Grenoble walnut", "Nutmeg", "Nutmeg powder", "Grated nutmeg", "Nutmeg", "Powdered hazelnut pack(s)", "Pine pinions", "Pistachios", "Chopped pistachios", "Unsalted pistachios", "Hazelnut powder", "Pistachio powder", "Almond powder", "Sesame", "White sesame"])
]

item_units = {
    "u_kg": ["Dried apricots", "Agar-agar", "Chopped almonds", "Vanilla almond powder", "Almonds", "Slivered almonds", "Vanilla slivered almonds", "Lamb collars", "Powdered almonds", "Pineapple", "Chopped dill", "Lemon vanilla flavor", "White asparagus", "Beet(s)", "Red beet(s)", "Butter", "Peanut butter", "Semi-salted butter", "Soft butter", "Salted butter", "Soft butter", "Biscuits", "Spoon cookies", "Bisques de homard", "Chicken breast(s)", "Chard(s)", "Boursin(s)", "Blue Bresse", "Brioche", "Burrata", "Beef", "Ground beef", "Cacahuètes", "Bitter cocoa", "Cocoa powder", "Unsweetened cocoa powder", "Soluble coffee", "Duck(s)", "Cancoillotte", "Cinnamon", "Brown sugar", "Cherries", "Walnut kernels", "Sausage meat", "Mushrooms", "Paris mushrooms", "Chanterelle(s)", "Chantilly", "Breadcrumbs", "Chocolate", "Bitter chocolate powder", "Milk chocolate", "White chocolate", "Dark chocolate", "50% dark chocolate", "Dark pastry chocolate", "70% cocoa dark chocolate", "Dark baking chocolate", "Pastry chocolate", "Chorizo", "Strong chorizo", "Brussels sprouts", "Green cabbage", "Cabbage(s)", "Chou(x) de bruxelle", "Cauliflower", "Chestnut(s)", "Goat cheese", "Heart of palm", "Quince", "Lamb collar", "Apple compote", "Comté cheese", "Shredded Comté cheese", "Tomato concentrate", "Conchiglionis", "Raspberry jam", "Chocolate chips", "Parmesan shavings", "Hulls", "Corn flakes", "Raspberry sauce", "Red fruit coulis", "Tomato coulis", "Butternut squash", "Couscous", "Crab(s)", "Crackers salés", "Dried cranberries", "Cranberrys", "Shrimp(s)", "Shrimp(s)", "Grey shrimps", "Pink shrimps", "Crème", "Light cream", "Custard cream", "Chestnut cream", "Whole cream", "30% full cream", "Whipping cream", "Crème fleurette 30%", "Whipped cream", "Heavy cream", "Fresh cream", "30% fresh cream", "30% fresh cream", "3% light cream", "Whole cream", "Mounted cream", "Sour cream", "Crêpe dentelle(s)", "Cumin", "Turmeric", "Curry", "Capers", "Porcini mushrooms", "Celery root", "Medjool date", "Dates", "Medjool date", "Dried dates", "Diced ham", "Emmental", "Grated Emmental cheese", "Squid(s)", "Fiscella", "Flour", "Buckwheat flour", "Wheat flour", "Rice flour", "Sticky rice flour", "Sticky rice flour", "Flour t55", "Sifted flour", "Feta cheese", "Figs", "Dried figs", "Beef tenderloin", "Pork tenderloin", "Pork tenderloin", "Salmon fillet(s)", "Herbs", "Oat flakes", "Foie gras", "Strawberry(s)", "Tagada strawberry(s)", "Raspberry(s)", "Cheese", "Cottage cheese", "0% fromage blanc", "Goat's cheese", "Feta cheese", "Grated cheese", "Starch", "Corn starch", "Gavotte", "Ginger", "Ginger powder", "Giraumon", "Clove(s)", "Chocolate ice cream", "Gnocchis", "Gorgonzola", "Chia seeds", "Pumpkin seeds", "Cumin seeds", "Sesame seeds", "White sesame seeds", "Golden sesame seeds", "Goose fat", "Coarse salt", "Gruyère", "Grated Gruyère", "Guanciale", "Poultry gizzard(s)", "Green bean(s)", "White beans", "Pink beans", "Kidney beans", "Green bean(s)", "Herbs", "Dried herbs", "Ham", "Cooked ham", "Parma ham", "Jeunes pousses d'épinard", "Ketchup", "Lambis", "Rabbit", "Smoked bacon", "Bacon", "Smoked bacon", "Lenses", "Coral lentils", "Green lentils", "Raw linguine", "Litchi(s)", "Macaronis", "Maïzena", "Mango(s)", "Cassava", "Margarine", "Melted margarine", "Vegetable margarine", "Mascarpone", "Mayonnaise", "Corn", "Maïzena", "McVitties", "Breadcrumbs", "Honey", "Truffle honey", "Mini carottes", "Mirabelles", "Mirin", "Dried morel(s)", "Salt cod", "Mussels", "Mousseron(s)", "Mustard", "Strong mustard", "Old-fashioned mustard", "Mozzarella", "Grated mozzarella", "Nutmeg", "Myrtilles", "4-spice blend", "Nesquik", "Hazelnuts", "Shelled hazelnuts", "Chopped hazelnuts", "Hazelnut powder", "Whole hazelnuts", "Walnuts", "Crushed nuts", "Grated coconut", "Grenoble walnut", "Scallops", "Nutmeg", "Noodles", "Nutella", "Cod eggs", "Yellow onion(s)", "Green olives", "Olives", "Black olives", "Green olives", "Oregano", "Stale bread", "Beef chuck", "Papaya", "Parmesan", "Grated Parmesan", "Sweet potato(s)", "Pecorino", "Pennes", "Chopped parsley", "Chopped parsley", "Chopped parsley", "Petits beurres", "Peas", "Philadelphia", "Pine pinions", "Espelette pepper", "Cayenne pepper", "Sweet pepper powder", "Chilli powder", "Guinea fowl", "Pistachios", "Chopped pistachios", "Unsalted pistachios", "Pizzoccheri", "Oyster mushroom(s)", "Pears in syrup", "Split peas", "Chickpeas", "Fish", "Lamb belly", "Bell pepper(s)", "Bintje potato", "Potatoes", "Pork", "Ground pork", "Pumpkin(s)", "Almond powder", "Cocoa powder", "Yellow curry powder", "Hazelnut powder", "Pistachio powder", "Almond powder", "Pousses d'épinard", "Bamboo shoot", "Soybean pods", "Pralinoise", "Prune(s)", "Dried prunes", "Tomato pulp", "Raspberry puree", "Mash mango", "Mash passion", "Mash tomato", "Hazelnut paste", "Pistachio paste", "Rice paste", "Pasta", "Pépites de chocolat", "Pépites de chocolat noir", "Peaches in syrup", "Quinoa", "Dried grapes", "Ras-el-hanout", "Reblochon", "Leftover meat", "Rhubarb", "Ricotta", "Rigatonis", "Rice", "Arborio rice", "Sticky rice", "Long rice", "Round rice", "Plain puffed rice", "Thaï rice", "Risotto rice", "Roquefort", "Rocket salad", "Brittany shortbreads", "Safran", "Philadelphia", "Sainte-maure de touraine", "Buckwheat", "Savory", "Chocolate sauce", "Soy sauce", "Tomato sauce", "Yakitori sauce", "Smoked sausage(s)", "Sausage", "Salmon", "Cuttlefish", "Saddle of lamb", "Semolina", "Fine semolina", "Shiitake(s)", "Soybeans", "Oat bran", "Soup", "Spaghettis", "Spéculoos", "Sugar", "Brown sugar", "Complete sugar", "Crystal sugar", "Cane sugar", "Powdered sugar", "Powdered sugar", "Brown sugar", "Semolina sugar", "Sesame", "Tagliatelles", "Tahini", "Ham heel", "Tapioca", "Tuna", "Tomate(s) pelée(s)", "Chopped tomatoes", "Dried tomatoes", "Jerusalem artichoke", "Black truffle(s)", "Smoked trout", "Vanilla powder", "Vergeoise blonde", "Chinese vermicelli", "Rice vermicelli", "Meat", "Beef", "Ground beef", "Lean beef", "Minced meat", "Poultry", "Yogurt", "Chopped shallot(s)", "Sweetener", "Pork shoulder", "Chopped spinach", "Spinach(s)", "Dehydrated garlic", "Garlic powder", "Roasted almonds", "Baies roses", "Baies", "Bicarbonate of soda", "Bulot(s)", "Bûches chèvre", "Bitter cocoa powder", "Coffee", "Cinnamon powder", "Caramel", "Salted butter caramel", "Caviar", "Chocolate powder", "Strawberry jam", "Chopped coriander", "Coriander powder", "Corned-beef", "Croutons", "Cumin powder", "Capers", "Spices", "Vanilla essence", "Chopped tarragon", "Vanilla extract", "Poultry fillets", "Edible flowers", "Orange blossom", "Salt flowers", "Red fruit", "Red fruit", "Trim", "Vanilla ice cream", "Fennel seeds", "Mustard seeds", "Harissa", "Provence herbs", "Philadelphia", "Merguez", "Acacia honey", "Morel(s)", "Chocolate mousse", "Nutmeg powder", "Grated nutmeg", "5-berry mix", "Butter hazelnut", "Roasted hazelnuts", "Nutmeg", "Nutmeg powder", "Grated nutmeg", "Nuoc mâm", "Clam(s)", "Paprika", "Smoked paprika", "Japanese pearls", "Truffle beads", "Curled parsley", "Sweet peas", "Pork belly", "Pepper", "White pepper", "Lebanese pepper", "Black pepper", "Porto", "Chicken", "Peanut paste", "Green curry paste", "Raviole dough", "Bolognese sauce", "Nuoc mam sauce", "Sausage(s)", "Salt", "Smen", "Blond sugar", "Grainy sugar", "White sesame", "Tabasco", "Cabbage", "Chocolate vermicelli", "Ground meat 5% (fat)", "Wasabi", "Orange zests"],
    "u_l": ["Beef broth", "Vegetable broth", "Poultry broth", "Coconut cream", "Liquid fresh cream", "Whole cream", "Semi-thick fresh cream", "Heavy cream", "15% liquid cream", "30% cream", "Light cream", "Orange blossom water", "Poultry stock", "Veal stock", "Poultry stock", "Fish stock", "Oil", "Olive oil", "Coconut oil", "Rapeseed oil", "Walnut oil", "Sesame oil", "Sunflower oil", "Vegetable oil", "Orange juice", "Lemon juice", "Milk", "Unsweetened condensed milk", "Almond milk", "Goat's milk", "Coconut milk", "Soy milk", "Semi-skimmed milk", "Whole milk", "Plant milk", "Skim milk", "Teriyaki sauce", "Agave syrup", "Maple syrup", "Liquid vanilla", "White wine", "Dry white wine", "Red wine", "Amaretto", "Armagnac", "Bitter almond aroma", "Vanilla flavor", "Broth", "Cognac", "Colombo", "Food coloring red", "Whipped cream", "Balsamic vinegar cream with toscoro berries", "35% liquid cream", "Thick cream", "Cuttlefish ink", "Yellow food coloring drops", "Red food coloring drop(s)", "Grand marnier", "Peanut oil", "Frying oil", "Lime juice", "Lychee juice", "Grapefruit juice", "Veal juice", "Rhum", "Dark rum", "Vermouth", "Madeira wine", "Yellow wine", "Rosé wine", "Vinegar", "Balsamic vinegar", "White balsamic vinegar", "White vinegar", "Spirit vinegar", "Cider vinegar", "Rice vinegar", "Red wine vinegar", "Sherry vinegar", "Whisky"],
    "u_value": ["Turkey cutlets", "Chicken cutlets", "Chicken wing(s)", "Victoria pineapple", "Andouillette(s)", "Eggplant(s)", "Avocado(s)", "Banana(s)", "Tray(s) of button mushrooms", "Cherry tomato tray(s)", "Nice apples", "Thick slices of farmhouse bread", "Spoon cookies", "White egg(s)", "White leek(s)", "Jar of pickled bell pepper(s)", "Vegetable broth cube", "Chicken broth", "Celery stalk(s)", "Brick(s) of liquid crème fraîche", "Brocoli(s)", "Fennel bulb", "Bâtons citronnelle", "Bâtons surimis", "Bâtonnets surimis", "Goat's cheese log", "Camembert", "Carottes", "Fresh square", "Carrés chocolat noir", "Chipolatas", "Mild chorizo", "White cabbage", "Kohlrabi", "Lime(s)", "Clove(s)", "Lamb collar(s)", "Combava", "Cucumber(s)", "Pickles", "Zucchini(s)", "Broth cube(s)", "Beef bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Vegetable bouillon cube(s)", "Rabbit legs", "Chicken thighs", "Lamb chop", "Artichoke heart", "Back of fresh salmon", "Endives", "Beef rib steak", "Shoulder(s) of lamb", "Turkey cutlet(s)", "Chicken cutlet(s)", "Star anise", "Fennel", "Brick pastry", "Gelatin sheets", "Lasagne sheet(s)", "Bay leaf(s)", "Filo sheets", "Rice leaves", "White figs", "Smoked tenderloin", "Beef tenderloin", "Tournedos filet", "Pike-perch fillets", "Sole fillets", "Pie crust", "Passion fruit(s)", "Patties", "Rice cakes", "Gambas", "Gouttes de tabasco", "Granny", "Grappes de tomates", "Egg yolk(s)", "Khakies", "Kiwi(s)", "Kubor", "Lobsters", "Langoustines", "Manicotti", "Mackerel(s)", "Maroilles", "Mini snickers", "Pieces of sugar", "Turnip(s)", "White onion(s)", "Sweet onion(s)", "Chopped onion(s)", "Oignon(s) moyen(s)", "Red onion(s)", "Green onion(s)", "Chopped onion(s)", "Pitted black olives", "Orange(s)", "Orange(s)", "Oreos", "Hamburger bun(s)", "Grapefruit", "Parsnips", "Boudoir package(s)", "Marshmallow pack(s)", "Wafer pack(s)", "Pack(s) of bacon", "Package(s) smoked bacon", "Powdered hazelnut pack(s)", "Paupiettes", "Rump steak(s)", "Salmon steak(s)", "Small peppers", "Small green peppers", "Small red peppers", "Small onion(s)", "Small apples", "Small tomatoes", "Pigeon", "Dried pepper", "Green pepper", "Red pepper(s)", "Dragon fruit", "Praline plate(s)", "Pear(s)", "Leek(s)", "Yellow bell pepper(s)", "Red bell pepper(s)", "Green bell pepper(s)", "Apple(s)", "Potato(es)", "Pumpkin", "Octopus(s)", "Pitted prune(s)", "Shortbread dough", "Pizza dough", "Pie dough", "Pâte(s) brisée(s)", "Peaches", "Quenelle(s)", "Monkfish tail(s)", "Raifort", "Goat's cheese log", "Rouleau pâte(s) feuilletée(s)", "Rumsteak(s)", "Scallops", "Pike-perch", "Sardine", "Sausage(s)", "Tuna steak(s)", "Chopped steak(s)", "Guinea fowl supreme", "Capon supremes", "Garlic and parsley tablet(s)", "Milk chocolate tablet(s)", "White chocolate tablet(s)", "Black chocolate tablet(s)", "Rhubarb stems", "Tomato(s)", "Tomate(s) cerise(s)", "Tomate(s) confite(s)", "Dried tomato(s)", "Medium tomatoes", "Corn tortilla", "Tortillas", "Slices of ham", "Slice(s) of Bayonne ham", "Parma ham slice(s)", "Slices of smoked salmon", "Trout(s)", "Twix", "Lemon zests", "Lemon zests", "Shallot(s)", "Minced shallot(s)", "Lamb shoulder", "Egg(s)", "Parsley bouquet", "Garnished bouquet(s)", "Bottle(s) brut champagne", "Bottle(s) red wine", "Can(s) tomato paste", "Lemon(s)", "Crêpes", "Toothpicks", "Onion(s)", "Milk bread", "Chili pepper", "Sweet pepper", "Hot pepper(s)", "Pâte feuilletée", "Garlic sausage"],
    "u_bunch": ["Green asparagus(s)", "Rosemary"],
    "u_slice": ["Bacon", "Baguette", "Cheddar", "Swordfish", "Leg of lamb", "Bread", "Brioche bread", "Gingerbread", "Country bread", "Crumb bread"],
    "u_bunch1": ["Chervil", "Menthe", "Parsley", "Flat-leaf parsley"],
    "u_leaf": ["Basil", "Gelatin", "Lettuce", "Lasagnes", "Laurel", "Fresh mint", "Filo dough", "Salad", "Sauge", "Wraps"],
    "u_sprig": ["Dill", "Thyme", "Chives", "Coriander", "Tarragon"],
    "u_box": ["Flageolet(s)", "Garlic and herb cheese", "Condensed milk", "Sweetened condensed milk", "Litchi in syrup", "Mutti garlic polpa"],
    "u_bag": ["Cream pudding", "Crêpes dentelles", "Sachet(s) de crêpes dentelles", "Instant jelly", "Yeast", "Baker's yeast", "Baking powder", "Frozen vegetable medley", "Vanilla sugar", "Tortellinis"],
    "u_pod": ["Garlic", "Cardamom", "Vanilla"]
}

for i, (category_name, items) in enumerate(item_categories):
    for item in items:
        #print(item)
        unit_info = find_table_containing_string(item_units, item)
        print(unit_info)

u_value
u_value
u_kg
u_kg
u_kg
u_kg
u_kg
u_kg
u_value
u_kg
u_value
u_kg
u_kg
u_kg
u_value
u_value
u_value
u_kg
u_value
u_value
u_value
u_value
u_kg
u_kg
u_kg
u_value
u_kg
u_kg
u_kg
u_l
u_l
u_l
u_slice
u_kg
u_kg
u_kg
u_kg
u_kg
u_kg
u_l
u_kg
u_kg
u_kg
u_kg
u_kg
u_kg
u_kg
u_value
u_value
u_value
u_value
u_value
u_kg
u_kg
u_kg
u_kg
u_kg
u_kg
u_kg
u_value
u_kg
u_kg
u_kg
u_kg
u_value
u_kg
u_value
u_value
u_value
u_kg
u_value
u_value
u_value
u_kg
u_kg
u_kg
u_kg
u_kg
u_kg
u_kg
u_value
u_kg
u_l
u_l
u_value
u_value
u_l
u_l
u_l
u_l
u_l
u_l
u_l
u_l
u_kg
u_l
u_l
u_kg
u_l
u_l
u_l
u_l
u_l
u_l
u_l
u_l
u_pod
u_kg
u_kg
u_kg
u_bunch
u_value
u_value
u_value
u_value
u_kg
u_kg
u_value
u_kg
u_value
u_value
u_value
u_value
u_kg
u_kg
u_kg
u_value
u_kg
u_value
u_kg
u_kg
u_kg
u_kg
u_kg
u_value
u_kg
u_value
u_value
u_kg
u_kg
u_value
u_kg
u_value
u_value
u_box
u_kg
u_kg
u_value
u_kg
u_kg
u_kg
u_kg
u_kg
u_leaf
u_kg
u_kg
u_kg
u_bag
u_kg
u_kg
u_kg
u_kg
u_value
u_value
u_value
u_value
u_value
u_kg
u_value
u_value
u_va